In [1]:
import numpy as np
from numpy import fft

import scipy
from scipy.optimize import newton, curve_fit
from scipy.ndimage import center_of_mass

from numba import njit
import glob

import matplotlib.pyplot as plt
import h5py
import yt

# Helper functions

## Data I/O

In [2]:
def extract_data(h5_filepath, dims, begin = 0, end = 1, nVars = 38):
    if type(dims) == list:
        dims = np.array(dims)

    with h5py.File(h5_filepath,'r') as h5f:
        keys = list(h5f.keys())
        test = h5f[keys[-1]]
        keys = list(test.keys())
        
        # boxDims = test[keys[1]][()]
        # # print(boxDims)
        domain_decomp = test[keys[1]][()]
        data = test[keys[2]][()]
    
    output = np.zeros((end - begin, *dims))
    decomp_box_size = [int(domain_decomp[0][5]-domain_decomp[0][2]+1),
                       int(domain_decomp[0][4]-domain_decomp[0][1]+1),
                       int(domain_decomp[0][3]-domain_decomp[0][0]+1)]
    sz = 1
    for i in decomp_box_size: sz *= i
    revdims = np.flip(dims)

    for i in range(begin, end):
        currVar = np.zeros(revdims)
        for j in range(0, len(domain_decomp)):
            curr_decomp = domain_decomp[j]
            slc = np.s_[curr_decomp[2]:curr_decomp[5]+1, curr_decomp[1]:curr_decomp[4]+1, curr_decomp[0]:curr_decomp[3]+1]
            currVar[slc] = data[(i + j*nVars)*sz:(i+1 + j*nVars)*sz].reshape(*decomp_box_size)
    
        output[i - begin] = np.moveaxis(currVar, [0, -1], [-1, 0])

    return output

## LBM and thermodynamic model

In [3]:
def lattice_fourier_laplacian(kx, ky, kz):
    expr1 = np.cos(kx) + np.cos(ky) + np.cos(kz)
    expr2 = np.cos(kx)*np.cos(ky) + np.cos(ky)*np.cos(kz) + np.cos(kx)*np.cos(kz)
    out = 2/9*expr1 + 2/9*expr2 - 4/3
    cs2 = 1/3
    return -out/cs2

class swift_et_al_1996_thermodynamic_model:
    def __init__(self, density = 1, C0 = 0, chi = 0.4, T = 0.25, kappa = 0.01):
        self.chi = chi
        self.T = T
        self.kappa = kappa
        self.rho = density
        self.C0 = C0

    def sound_speed_square(self):
        out = self.T
        return out

    def cs2k(self, kx = 0, ky = 0, kz = 0):
        thermal_cs2 = self.sound_speed_square()
        k2 = lattice_fourier_laplacian(kx, ky, kz)
        out = thermal_cs2 + k2*self.kappa
        return out
    
    def calculate_df_dphi(self):
        rho = self.rho
        phi = self.C0
        chi = self.chi
        T   = self.T
        out = -chi/2.*(phi/rho) + T/2.*np.log((1. + phi/rho)/(1. - phi/rho))
        return out

    def calculate_dmup_drho(self):
        rho = self.rho
        phi = self.C0
        chi = self.chi
        T   = self.T
        out = -T*phi/(rho**2 - phi**2) + chi/2*phi/(rho**2)
        return out

    def calculate_dmup_dphi(self):
        rho = self.rho
        phi = self.C0
        chi = self.chi
        T   = self.T
        out = T*rho/(rho**2 - phi**2) - chi/(2*rho)
        return out
    
    def mu_ck(self, kx = 0, ky = 0, kz = 0):
        ref_state = self.calculate_dmup_dphi()
        k2 = lattice_fourier_laplacian(kx, ky, kz)
        out = ref_state + k2*self.kappa
        return out

In [4]:
def fb(phi, l, T, rho = 1):
    model = swift_et_al_1996_thermodynamic_model(rho, phi, l, T, kappa = 0)
    out = model.calculate_df_dphi()
    return out

def ih_direct(profile):
    grad_field_dx = np.abs(np.gradient(profile, axis = 0))
    idx_max = np.argmax(grad_field_dx, axis = 0)
    return idx_max

def ih_profile_fit(profile, chi, T, kappa):
    nx, ny, nz = profile.shape
    out = np.zeros((ny, nz))
    phi0 = -newton(fb, x0 = (0.6), args = (chi, T))
    xi = np.sqrt((0.142*kappa)/((chi/2 - T)-0.31*T*(phi0**2)))
    fit_func = lambda x, b:phi0*np.tanh(0.76*(x - b)/xi)

    xraw = np.arange(0, nx, 1)

    for y in range(ny):
        for z in range(nz):
            slc = profile[:, y, z]
            popt, pcov = curve_fit(fit_func, xraw, slc, p0 = [nx//2])
            out[y, z] = popt[0]
    return out

def interface_height(density, chi, T, kappa, method = "direct"):
    height_func = np.zeros(density.shape[1:])

    nx, ny, nz = density.shape
    lo_min = nx//4
    hi_max = 3*(nx//4)
    zero_factor = (nx/2 - 1)/2
    yraw = density[lo_min:hi_max, :, :]

    if method == 'direct':
        height_func = ih_direct(yraw) - zero_factor
    elif method == "profile_fit":
        height_func = ih_profile_fit(yraw, chi, T, kappa) - zero_factor
    else:
        raise ValueError(f'{method} is invalid to calculate interface height')
    
    return height_func

## Plotting

In [5]:
def spherically_averaged_structure_factor(data, thermo_model, scale_factor = 1, func = None, shift = True, cs = True):
    L = min(data.shape)
    S = data.copy()

    if shift:
        freqs = fft.fftshift(fft.fftfreq(L))
    else:
        freqs = fft.fftfreq(L)
    if len(data.shape) == 3:
        kx, ky, kz = np.meshgrid(*tuple([2*np.pi*freqs for L in [L, L, L]]), indexing='ij')
        k = np.stack([kx, ky, kz], axis = -1)
    elif len(data.shape) == 2:
        kx, ky = np.meshgrid(*tuple([2*np.pi*freqs for L in [L, L]]), indexing='ij')
        k = np.stack([kx, ky], axis = -1)
     
    k1 = np.linalg.norm(k, axis=-1).flatten()
    
    if func is not None:
        if cs:
            S = func(S, thermo_model.cs2k(kx, ky, kz))
        else:
            S = func(S, thermo_model.mu_ck(kx, ky, kz))
    S /= scale_factor

    # test[slc] /= test.sum()
    # S[L//2, L//2, L//2] /= S.sum()

    S1 = S.flatten()
    kmin = 2*np.pi/L # sampling frequency
    where = np.s_[:]#np.where(k1<=kmax)
    bins = np.arange(L//2+1)*kmin # kmax+1 for bin_edges: len(bins)=len(hist)+1
    
    shells = np.histogram(k1[where], bins, weights=S1[where])[0]
    counts = np.histogram(k1[where], bins)[0]
    return (bins[:-1]+bins[1:])/2, shells/counts

In [6]:
def cart2sph(x,y,z):
    azimuth = np.arctan2(y,x)
    elevation = np.arctan2(z,np.sqrt(x**2 + y**2))
    r = np.sqrt(x**2 + y**2 + z**2)
    return r, azimuth, elevation

def sph2cart(azimuth,elevation,r):
    x = r * np.cos(elevation) * np.cos(azimuth)
    y = r * np.cos(elevation) * np.sin(azimuth)
    z = r * np.sin(elevation)
    return x, y, z

def radial_equilibration(data, thermo_model, radius = 1, scale_factor = 1, func = None, cs = True):
    S = data.copy()
    L = min(S.shape)
    freqs = fft.fftshift(fft.fftfreq(L))
    kx, ky, kz = np.meshgrid(*tuple([2*np.pi*freqs for L in [L, L, L]]), indexing='ij')

    r, t, p = cart2sph(kx, ky, kz)

    if func is not None:
        if cs:
            S = func(S, thermo_model.cs2k(kx, ky, kz))
        else:
            S = func(S, thermo_model.mu_ck(kx, ky, kz))
    S /= scale_factor
    # S[L//2, L//2, L//2] /= S.sum()
    
    idxs = np.isclose(r, radius, atol = 2*np.pi/L)
    t = t[idxs]
    p = p[idxs]
    out = S[idxs]

    return t, p, out

In [7]:
def make_bins(to_bin1, binsize, to_bin2 = None):
    bins = np.linspace(to_bin1.min(), to_bin1.max(), binsize)

    out1 = np.zeros(binsize)
    shell = np.digitize(to_bin1, bins = bins, right = True)
    np.add.at(out1, shell, to_bin1)
    unique, counts = np.unique(shell, return_counts=True)
    out1 = out1[unique]
    out1 /= counts

    if to_bin2 is None:
        return bins, out1
    else:
        out2 = np.zeros(binsize)
        np.add.at(out2, shell, to_bin2)
        unique, counts = np.unique(shell, return_counts=True)
        out2 = out2[unique]
        out2 /= counts
        return out1, out2

## Droplets

In [8]:
def droplet_radius(density, Vp = 0, np_sphere = 0, rho_sphere = 1):
    if isinstance(density, int):
        return np.nan
    else:
        center = tuple([ l//2 for l in density.shape ])
        rho_d = density[center]
        rho_m = density[0,0,0]
        mass = np.sum(density - rho_m) + 0.5*Vp*np_sphere*rho_sphere
        R = (3./4./np.pi*mass/(rho_d-rho_m))**(1./3.)
        return R

def pressure_jump(pressure):
    center = tuple([ l//2 for l in pressure.shape ])
    dP = pressure[center] - pressure[0,0,0]
    return dP

def inertia_tensor(cm,OutArray):
    ind = np.transpose(np.indices(OutArray.shape), axes=(1,2,3,0))
    pos = ind - cm
    r2 = np.einsum('ijkl,ijkl->ijk',pos,pos)          # inner product
    rr = np.einsum('ijkm,ijkn->ijkmn',pos,pos)        # outer product
    r2 = np.einsum('ijk,mn->ijkmn',r2,np.identity(3)) # multiply with unit matrix
    I = np.einsum('ijk,ijkmn->mn',OutArray,r2-rr)     # sum m*(r2-rr)
    return I

def droplet_mass(OutArray):
    sum = np.sum(OutArray)
    return sum

def eigenvalues(gyration_matrix):
    eigen = np.linalg.eigh(gyration_matrix)
    values = eigen[0]
    values.sort()
    return values

def radii_pca(eigvals, mass):
    a = np.sqrt((5/(2*mass))*(eigvals[1]+eigvals[2]-eigvals[0]))
    b = np.sqrt((5/(2*mass))*(eigvals[0]+eigvals[2]-eigvals[1]))
    c = np.sqrt((5/(2*mass))*(eigvals[0]+eigvals[1]-eigvals[2]))

    return [a, b, c]

def droplet_fluctuations(fluctuations, temp = 1e-7):
    reps, dims = fluctuations.shape
    sums = np.zeros((reps, dims))
    difs = np.zeros((reps, dims))
    idx = 0

    for i in range(0, dims - 1):
        for j in range(i + 1, dims):
            sums[:, idx] = np.power(fluctuations[:, i] + fluctuations[:, j], 2)
            difs[:, idx] = np.power(fluctuations[:, i] - fluctuations[:, j], 2)
            idx += 1
    
    sums = np.mean(sums, axis = 0)
    difs = np.mean(difs, axis = 1)
            
    sums = np.mean(sums)
    difs = np.mean(difs)

    y20 = 15*temp/(16*np.pi*sums)
    y22 = 45*temp/(16*np.pi*difs)

    return [y20, y22]

# Validation

## Homogeneous system

In [9]:
L = 64
boxDims = [L, L, L]
use_hdf5 = False

chi = 0.4
T = 0.25
kappa = 0.01
kbt = 1e-7

rho0 = 1.0
phi0 = 0.0
thermo_vars = swift_et_al_1996_thermodynamic_model(rho0, phi0, chi, T, kappa)

noise_type = "spatially_independent"
# noise_type = "spatially_dependent"
save_dir = f"validation/equilibration_tests/{noise_type}"
# save_dir = "./"

freqs = fft.fftshift(fft.fftfreq(L))
if "in" in noise_type:
    kx, ky, kz = [0, 0, 0]
else:
    kx, ky, kz = np.meshgrid(*tuple([2*np.pi*freqs for L in [L, L, L]]), indexing='ij')

In [ ]:
if use_hdf5:
    paths = sorted(glob.glob(f"{save_dir}/hydro*.h5"))[1:]

    S_rho = np.zeros(([L]*3), dtype = complex)
    S_phi = np.zeros([L]*3, dtype = complex)
    S_v = np.zeros((*[L]*3,3,3), dtype = complex)

    for i, h5_filepath in enumerate(paths):
        conserved_moments = extract_data(h5_filepath, boxDims, begin = 0, end = 5, nVars = 38)

        var = conserved_moments[0]
        vark = fft.fftn(var)
        S_rho += vark*np.conj(vark)

        var = conserved_moments[1]
        vark = fft.fftn(var)
        S_phi += vark*np.conj(vark)

        var = conserved_moments[2:]
        vark = fft.fftn(var, axes = [1, 2, 3])
        S_v += np.einsum('i..., j...->...ij',vark, np.conj(vark))


    S_rho /= (len(paths)*L**3*kbt)
    S_phi /= (len(paths)*L**3*kbt)
    S_v /= (len(paths)*L**3*kbt)

    data_rho = [S_rho.real]
    data_phi = [S_phi.real]
    data_v = [S_v[..., 0, 0].real, S_v[..., 1, 1].real, S_v[..., 2, 2].real]
else:
    ts = yt.load(f"{save_dir}/SF_plt_mag*")
    ds = ts[-1]
    ad = ds.all_data()

    data_rho = [np.array(ad[('boxlib', 'struct_fact_density_density')]).reshape(boxDims)/kbt]
    data_phi = [np.array(ad[('boxlib', 'struct_fact_phi_phi')]).reshape(boxDims)/kbt]
    data_v = [np.array(ad[('boxlib', 'struct_fact_ux_ux')]).reshape(boxDims)/kbt, 
              np.array(ad[('boxlib', 'struct_fact_uy_uy')]).reshape(boxDims)/kbt,
              np.array(ad[('boxlib', 'struct_fact_uz_uz')]).reshape(boxDims)/kbt]

    # ds.field_list

### Density

In [ ]:
data = data_rho
experiment_muck = thermo_vars.cs2k(kx, ky, kz)

test = data[0].copy()
# test /= kbt

fig, axs = plt.subplots(1, 3, figsize = (9, 3))

ax = axs[0]
test *= (experiment_muck)
# test[slc] /= test.sum()
im = ax.imshow(test[L//2, :, :])
plt.colorbar(im, ax = ax, shrink = 0.8)

ax = axs[1]
im = ax.imshow(test[:, L//2, :])
plt.colorbar(im, ax = ax, shrink = 0.8)

ax = axs[2]
im = ax.imshow(test[:, :, L//2])
plt.colorbar(im, ax = ax, shrink = 0.8)

fig.tight_layout()

In [ ]:
slc = np.s_[L//2, L//2, L//2]

ar = 1.5
sz = 3
binsize = 8
fig, ax = plt.subplots(1, 1, figsize = (sz*ar, sz))

colors = ['b', "r", "k"]
markers = ["o", "s", '^']
labels = [r"$\rho$"]

rho_scale = lambda a, c: a*c
scale_factor = 1

for d in data:
        currData = d.copy()
        # currData /= kbt
        x, y = spherically_averaged_structure_factor(currData, thermo_vars, scale_factor = 1, func = rho_scale)
        y[0] = 1
        x, y = make_bins(x, binsize, to_bin2 = y)
        ax.plot(x, y, 
                marker = markers[0], color = colors[0],
                markerfacecolor = "None", markersize = 10, 
                linestyle = "None", label = labels[0])

ax.plot(x, [1]*x.size, 'lime', lw = 2)

ax.set_xlabel(r"$k$", fontsize = 14)
ax.set_ylabel(r"$\frac{ \langle | \rho(k) | ^2 \rangle }{S(k)}$", fontsize = 14)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=10)
ax.legend()
ax.set_ylim(bottom = 0.95)
fig.tight_layout()
# fig.savefig("./equilibration_ratio.svg")

In [ ]:
sz = 4
fig, tax = plt.subplots(1, 1, figsize = (sz, sz), subplot_kw={'projection': 'polar'})
fig, pax = plt.subplots(1, 1, figsize = (sz, sz), subplot_kw={'projection': 'polar'})

colors = ['b', "r", "k"]
markers = ["o", "s", '^']

radii = [0.5, 1.5, 2.5]

t_angle = 0
p_angle = 0 
binsize = 16

# rho_scale = None
# scale_factor = 1/cs2

for d in data:
    ax1 = tax
    ax2 = pax
    currData = d.copy()
    # currData /= kbt
    for i, r in enumerate(radii):
        # t, p, sf = radial_equilibration(d, rho0, phi0, radius = r, func=rho_scale, scale_factor = scale_factor)
        t, p, sf = radial_equilibration(currData, thermo_vars, radius = r, func=rho_scale, scale_factor = scale_factor)

        idxs = np.isclose(np.abs(t), t_angle/180*np.pi, atol = np.pi/L)
        angles = np.abs(p)[idxs]
        segment = sf[idxs]
        angles, segment =  make_bins(angles, binsize, to_bin2 = segment)
        ax1.plot(angles, segment, color = colors[i], label = f"R = {r}",lw = 2)

        idxs = np.isclose(np.abs(p), p_angle/180*np.pi, atol = np.pi/L)
        angles = np.abs(t)[idxs]
        segment = sf[idxs]
        angles, segment =  make_bins(angles, binsize, to_bin2 = segment)
        ax2.plot(angles, segment, color = colors[i], label = f"R = {r}",lw = 2)


    ax1.set_rticks([0.5, 1.0])
    ax1.set_thetalim([0, np.pi/2])
    ax1.set_ylabel(r"$\frac{ \langle | \rho(k) | ^2 \rangle }{S(k)}$", fontsize = 14)
    ax1.legend(loc = 'upper right', bbox_to_anchor=(1.1, 1.1), fancybox=True)
    ax1.set_title(r"$\theta = {{{0}}}^{{\circ}}$".format(t_angle))

    ax2.set_rticks([0.5, 1.0])
    ax2.set_thetalim([0, np.pi])
    ax2.set_ylabel(r"$\frac{ \langle | \rho(k) | ^2 \rangle }{S(k)}$", ha = 'right', va = "top", fontsize = 14)
    ax2.legend(loc = 'upper right', bbox_to_anchor=(1.1, 1.1), fancybox=True)
    ax2.set_title(r"$\psi = {{{0}}}^{{\circ}}$".format(p_angle))

fig.tight_layout()

### C1

In [ ]:
data = data_phi

# freqs = fft.fftshift(fft.fftfreq(L))
# kx, ky, kz = np.meshgrid(*tuple([2*np.pi*freqs for L in [L, L, L]]), indexing='ij')
# k2 = lattice_fourier_laplacian(kx, ky, kz)
experiment_muck = thermo_vars.mu_ck(kx, ky, kz)

test = data[0].copy()
# test[slc] = (test.sum() - test[slc])/(L**3)

fig, axs = plt.subplots(1, 3, figsize = (9, 3))

ax = axs[0]
# test[slc] = (test.sum() - test[slc])/32**3
test *= (experiment_muck)
# test /= kbt
im = ax.imshow(test[L//2, :, :])
plt.colorbar(im, ax = ax)

ax = axs[1]
im = ax.imshow(test[:, L//2, :])
plt.colorbar(im, ax = ax)

ax = axs[2]
im = ax.imshow(test[:, :, L//2])
plt.colorbar(im, ax = ax)

fig.tight_layout()

In [ ]:
ar = 1.5
sz = 3
fig, ax = plt.subplots(1, 1, figsize = (sz*ar, sz))

colors = ['b', "r", "k"]
markers = ["o", "s", '^']
labels = [r"$\phi$"]

rho_func = lambda a, c: a*c
scale_factor = 1
binsize = 8

for d in data:
        currData = d.copy()
        # currData /= kbt
        x, y = spherically_averaged_structure_factor(currData, thermo_vars, scale_factor = scale_factor, func = rho_func, cs = False)
        x, y = make_bins(x, binsize, to_bin2 = y)

        y[0] = 1
        ax.plot(x, y, 
                marker = markers[0], color = colors[0],
                markerfacecolor = "None", markersize = 10, 
                linestyle = "None", label = labels[0])

ax.plot(x, [1]*x.size, 'lime', lw = 2)

ax.set_xlabel(r"$k$", fontsize = 14)
ax.set_ylabel(r"$\frac{ \langle | \phi(k) | ^2 \rangle }{S(k)}$", fontsize = 14)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=10)
ax.legend()
# print(x, y)
ax.set_ylim(bottom = 0.95)

In [ ]:
sz = 4
fig, tax = plt.subplots(1, 1, figsize = (sz, sz), subplot_kw={'projection': 'polar'})
fig, pax = plt.subplots(1, 1, figsize = (sz, sz), subplot_kw={'projection': 'polar'})

colors = ['b', "r", "k"]
markers = ["o", "s", '^']

radii = [0.5, 1.5, 2.5]

t_angle = 0
p_angle = 0 
binsize = 16

# rho_scale = None
# scale_factor = 1/cs2

for d in data:
    ax1 = tax
    ax2 = pax
    currData = d.copy()
    # currData /= kbt
    for i, r in enumerate(radii):
        t, p, sf = radial_equilibration(currData, thermo_vars, radius = r, func=rho_scale, scale_factor = scale_factor, cs = False)

        idxs = np.isclose(np.abs(t), t_angle/180*np.pi, atol = np.pi/L)
        angles = np.abs(p)[idxs]
        segment = sf[idxs]
        angles, segment =  make_bins(angles, binsize, to_bin2 = segment)
        ax1.plot(angles, segment, color = colors[i], label = f"R = {r}",lw = 2)

        idxs = np.isclose(np.abs(p), p_angle/180*np.pi, atol = np.pi/L)
        angles = np.abs(t)[idxs]
        segment = sf[idxs]
        angles, segment =  make_bins(angles, binsize, to_bin2 = segment)
        ax2.plot(angles, segment, color = colors[i], label = f"R = {r}",lw = 2)


    ax1.set_rticks([0.5, 1.0])
    ax1.set_thetalim([0, np.pi/2])
    ax1.set_ylabel(r"$\frac{ \langle | \phi(k) | ^2 \rangle }{S(k)}$", fontsize = 14)
    ax1.legend(loc = 'upper right', bbox_to_anchor=(1.1, 1.1), fancybox=True)
    ax1.set_title(r"$\theta = {{{0}}}^{{\circ}}$".format(t_angle))

    ax2.set_rticks([0.5, 1.0])
    ax2.set_thetalim([0, np.pi])
    ax2.set_ylabel(r"$\frac{ \langle | \phi(k) | ^2 \rangle }{S(k)}$", ha = 'right', va = "top", fontsize = 14)
    ax2.legend(loc = 'upper right', bbox_to_anchor=(1.1, 1.1), fancybox=True)
    ax2.set_title(r"$\psi = {{{0}}}^{{\circ}}$".format(p_angle))

fig.tight_layout()

### Velocities

In [ ]:
data = data_v
slc_s = [np.s_[L//2, :, :], np.s_[:, L//2, :], np.s_[:, :, L//2]]
labels = ['yz', 'xz', 'xy']
fig, axs = plt.subplots(3, 3, figsize = (9, 9))

for i in range(3):
    d = data[i].copy()
    # d /= kbt
    # d[slc] = (d.sum() - d[slc])/(L**3)
    for j in range(3):
        ax = axs[i, j]
        im = ax.imshow(d[slc_s[j]], vmin = 0.8, vmax = 1.2)
        plt.colorbar(im, ax = ax, shrink = 0.8)
        ax.set_title(f"$S_{{u_{chr(120+i)}u_{chr(120+i)}}}$ {labels[j]}")

fig.tight_layout()

In [ ]:
ar = 1.5
sz = 3
fig, ax = plt.subplots(1, 1, figsize = (sz*ar, sz))
binsize = 8

colors = ['b', "r", "k"]
markers = ["o", "s", '^']
labels = [r"$u_x$", r"$u_y$", r"$u_z$"]

for i, d in enumerate(data):
        currData = d.copy()
        # currData /= kbt
        x, y = spherically_averaged_structure_factor(currData, thermo_vars, scale_factor = 1)
        y[0] = 1
        x, y = make_bins(x, binsize, to_bin2 = y)

        ax.plot(x, y, 
                marker = markers[i], color = colors[i],
                markerfacecolor = "None", markersize = 10, 
                linestyle = "None", label = labels[i])
ax.plot(x, [1]*x.size, 'lime', lw = 2)

ax.set_xlabel(r"$k$", fontsize = 14)
ax.set_ylabel(r"$\frac{ \langle | u_{\alpha}(k) | ^2 \rangle }{\rho_{0}k_b T}$", fontsize = 14)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=10)
ax.legend()
ax.set_ylim(bottom = 0.95)

In [ ]:
sz = 4
fig, tax = plt.subplots(1, 3, figsize = (sz*3, sz), subplot_kw={'projection': 'polar'})
fig, pax = plt.subplots(1, 3, figsize = (sz*3, sz), subplot_kw={'projection': 'polar'})

colors = ['b', "r", "k"]
markers = ["o", "s", '^']

radii = [0.5, 1, 1.5]

t_angle = 0
p_angle = 0 
binsize = 16

for j, d in enumerate(data):
    ax1 = tax[j]
    ax2 = pax[j]

    currData = d.copy()
    # currData /= kbt
    for i, r in enumerate(radii):
        # t, p, sf = radial_equilibration(d, rho0, phi0, radius = r)
        t, p, sf = radial_equilibration(currData, thermo_vars, radius = r)

        idxs = np.isclose(np.abs(t), t_angle/180*np.pi, atol = np.pi/L)
        angles = np.abs(p)[idxs]
        segment = sf[idxs]
        angles, segment =  make_bins(angles, binsize, to_bin2 = segment)
        ax1.plot(angles, segment, color = colors[i], label = f"R = {r}",lw = 2)

        idxs = np.isclose(np.abs(p), p_angle/180*np.pi, atol = np.pi/L)
        angles = np.abs(t)[idxs]
        segment = sf[idxs]
        angles, segment =  make_bins(angles, binsize, to_bin2 = segment)
        ax2.plot(angles, segment, color = colors[i], label = f"R = {r}",lw = 2)

    ax1.set_rticks([0.5, 1.0])
    ax1.set_thetalim([0, np.pi/2])
    ax1.set_ylabel(r"$\frac{ \langle | u_{\alpha} | ^2 \rangle }{\rho_0 k_B T}$", fontsize = 14)
    ax1.set_title(r"$\theta = {{{0}}}^{{\circ}}$".format(t_angle))

    ax2.set_rticks([0.5, 1.0])
    ax2.set_thetalim([0, np.pi])
    ax2.set_ylabel(r"$\frac{ \langle | u_{\alpha} | ^2 \rangle }{\rho_0 k_B T}$", fontsize = 14, ha = 'right', va = "top", rotation = 'horizontal')
    ax2.set_title(r"$\psi = {{{0}}}^{{\circ}}$".format(p_angle))

ax1.legend(loc = 'upper right', bbox_to_anchor=(1.1, 1.1), fancybox=True)
ax2.legend(loc = 'upper right', bbox_to_anchor=(1.1, 1.1), fancybox=True)

fig.tight_layout()

## Interfacial fluctuations

### Convolutions

In [ ]:
L = 32
# dims = [L, L]
# slc = np.s_[:L, :L]

dims = [L, L, L]
slc = np.s_[:L, :L, 0]

idxs = np.indices(dims)
## Creating hydrovars parameters ##
rho = np.ones(dims)
phi = np.zeros(dims)
## Creating hydrovars parameters ##

## Creating inhomogenous system ##
phi[idxs[0] >= L//2 + 1] = 0.5
phi[idxs[0] <= L//2 - 1] = -0.5
## Creating inhomogenous system ##

## Convolving FFT with kernel ##
kernel = np.ones(dims)
rhok = fft.fftn(rho)
phik = fft.fftn(phi)
# mode = "full"
mode = "same"
rho_convolve = scipy.signal.convolve(rhok, kernel, mode = mode)/np.prod(dims)
phi_convolve = scipy.signal.convolve(phik, kernel, mode = mode)/np.prod(dims)
## Convolving FFT with kernel ##

## Multiplying convolution with its adjoint ##
complex_adjoint = rho_convolve.conj()
rho_convolve = rho_convolve*complex_adjoint
complex_adjoint = phi_convolve.conj()
phi_convolve = phi_convolve*complex_adjoint
## Multiplying convolution with its adjoint ##

sz = 4
ar = 1
row = 2
col = 4
fig = plt.figure(1, figsize=(sz*col*ar, sz*row))
# numbers are in row-col-figure with figure number being row major

fig.add_subplot(241)
plt.imshow(rho[slc])
plt.colorbar()
plt.title(r"$\rho(r)$")

fig.add_subplot(245)
plt.imshow(phi[slc])
plt.colorbar()
plt.title(r"$\phi(r)$")

fig.add_subplot(242)
plt.imshow(rhok.real[slc])
plt.colorbar()
plt.title(r"$\rho(k)$")

fig.add_subplot(246)
plt.imshow(phik.real[slc])
plt.colorbar()
plt.title(r"$\phi(k)$")

fig.add_subplot(243)
plt.imshow(rho_convolve[slc].real, vmin = 1, vmax = 1)
plt.colorbar()
plt.title(r"$real(\rho_k^{conv})$")

fig.add_subplot(247)
plt.imshow(phi_convolve[slc].real, vmin = phi_convolve.real[slc].min(), vmax = phi_convolve.real[slc].max())
plt.colorbar()
plt.title(r"$real(\phi_k^{conv})$")

fig.add_subplot(244)
plt.imshow(rho_convolve[slc].imag, vmin = 0, vmax = 0)
plt.colorbar()
plt.title(r"$imag(\rho_k^{conv}$)")

fig.add_subplot(248)
plt.imshow(phi_convolve[slc].imag, vmin = 0, vmax = 0)
plt.colorbar()
plt.title(r"$imag(\phi_k^{conv}$)")

fig.tight_layout()
print(rho_convolve.shape)

#### 1D case

1D convolutions have the following mathematical expression in the discrete case

$$ y(n) = \sum x(k)h(n - k)$$

Where x is the input function and h is the kernel upon which we are constructing our convolution. Lets expand the expression for what the first 3 convolutions of $y(n)$ will be

$$y(0) = x[0]h[0 - 0] + x[1]h[0 - 1] + x[2]h[0 - 2] ...$$
$$y(0) = x[0]h[0]$$

For $n = 0$, we see that once the index in h goes below 0, we trim off the terms. Expanding this to subsequent terms of $y$,

$$y(1) = x[0]h[1 - 0] + x[1]h[1 - 1] + x[2]h[1 - 2] ...$$
$$y(0) = x[0]h[1] + x[1]h[0]$$

Therefore, an algorithm for setting up a 1D convolution would just be tracking the indices for $x$ and $h$, resulting in pseudocode that looks like

```
1. Generate the output array of len(input)
2. Iterate through the indices i for input to begin iterating through n
3. Iterate through the kernel j to perform the summation
4. Once the difference between i and j is less than 0, stop adding to the list
```

In [ ]:
def conv1d_loop(input, kernel, bias = 0):
    output_size = input.size
    output = np.zeros(output_size)

    for i in range(0, output.size):
        sum = 0
        for j in range(0, i + 1):
            # print(j, i - j)
            sum += input[j]*kernel[j - i] + bias
        output[i] = sum
 
    return output

dims = [5]

input = np.random.randint(0, 5, size = dims)
filter = np.ones_like(input)

loop_test = conv1d_loop(input.flatten(), filter.flatten())
scipy_test = scipy.signal.convolve(input, filter, mode = "full").flatten()

slc = np.s_[:dims[0]]
np.isclose(loop_test - scipy_test[slc], 0, atol = 1e-7)

#### 2D case

2D convolutions have the following mathematical expression in the discrete case

$$ y(n) = \sum_{j}\sum_{i} x(i, j)h(m - i, n - j)$$

Where x is the input function and h is the kernel upon which we are constructing our convolution. Unlike the 1D case, we will implement 2 strategies for how the convolution will be done. In numpy and scipy notation, this is termed the `full` and the `same` convolution construction. For the `full` construction, we obtain an output array that is `xrow + hrow - 1` in row width, and `xcol + hcol - 1` in column height. For the `valid` construction, we obtain an output array that is `xrow` in row width and `xcol` in column height, the same size as the input array.

Lets expand the expression for what the first 3 convolutions of $y(n)$ will be

$$y(0) = x[0]h[0 - 0] + x[1]h[0 - 1] + x[2]h[0 - 2] ...$$
$$y(0) = x[0]h[0]$$

For $n = 0$, we see that once the index in h goes below 0, we trim off the terms. Expanding this to subsequent terms of $y$,

$$y(1) = x[0]h[1 - 0] + x[1]h[1 - 1] + x[2]h[1 - 2] ...$$
$$y(0) = x[0]h[1] + x[1]h[0]$$

Therefore, an algorithm for setting up a 1D convolution would just be tracking the indices for $x$ and $h$, resulting in pseudocode that looks like

```
1. Generate the output array of len(input)
2. Iterate through the indices i for input to begin iterating through n
3. Iterate through the kernel j to perform the summation
4. Once the difference between i and j is less than 0, stop adding to the list
```

In [ ]:
def conv_2d_loop_same(input, kernel, output):
    row, col = input.shape
    krow, kcol = kernel.shape
    kcenterx = krow//2
    kcentery = kcol//2
    for i in range(0, output.shape[0]):
        for j in range(0, output.shape[1]):
            for m in range(0, krow):
                for n in range(0, kcol):
                    ii = i + (m - kcenterx)
                    jj = j + (n - kcentery)
                    if (ii >= 0 and ii < krow and jj >= 0 and jj < kcol):
                            output[i, j] += input[ii, jj]*kernel[m, n]

def conv_2d_loop_full(input, kernel, output):
    row, col = input.shape
    krow, kcol = kernel.shape

    for i in range(0, output.shape[0]):
        for j in range(0, output.shape[1]):
            for m in range(0, krow):
                for n in range(0, kcol):
                    if (i - m >= 0) and (i - m < row) and (j-n >= 0) and (j-n < col):
                        output[i,j] = output[i,j] + kernel[m,n]*input[i-m,j-n]

def convolution_2D(input, kernel, mode = "full"):
    row, col = input.shape
    krow, kcol = kernel.shape

    if mode == "same":
        newRows = row
        newCols = col
        output = np.zeros((newRows, newCols))
        conv_2d_loop_same(input, kernel, output)
    else:
        newRows = row + krow - 1
        newColumns = col + kcol - 1
        output = np.zeros((newRows, newColumns))
        conv_2d_loop_full(input, kernel, output)
    
    return output

dims = [10, 10]
slc = np.s_[:dims[0], :dims[1]]

input = np.zeros(dims)
input[0, 0] = np.prod(dims)

filter = np.ones_like(input)

loop_test = convolution_2D(input, filter, mode = "full")
scipy_test = scipy.signal.convolve(input, filter, mode = "full")
print("mode=full match:", np.unique(np.isclose(loop_test[slc] - scipy_test[slc], 0, atol = 1e-7))[0])

loop_test = convolution_2D(input, filter, mode = "same")
scipy_test = scipy.signal.convolve(input, filter, mode = "same")
print("mode=same match:", np.unique(np.isclose(loop_test - scipy_test, 0, atol = 1e-7))[0])

#### 3D case

2D convolutions have the following mathematical expression in the discrete case

$$ y(n) = \sum_{j}\sum_{i} x(i, j)h(m - i, n - j)$$

Where x is the input function and h is the kernel upon which we are constructing our convolution. Unlike the 1D case, we will implement 2 strategies for how the convolution will be done. In numpy and scipy notation, this is termed the `full` and the `same` convolution construction. For the `full` construction, we obtain an output array that is `xrow + hrow - 1` in row width, and `xcol + hcol - 1` in column height. For the `valid` construction, we obtain an output array that is `xrow` in row width and `xcol` in column height, the same size as the input array.

Lets expand the expression for what the first 3 convolutions of $y(n)$ will be

$$y(0) = x[0]h[0 - 0] + x[1]h[0 - 1] + x[2]h[0 - 2] ...$$
$$y(0) = x[0]h[0]$$

For $n = 0$, we see that once the index in h goes below 0, we trim off the terms. Expanding this to subsequent terms of $y$,

$$y(1) = x[0]h[1 - 0] + x[1]h[1 - 1] + x[2]h[1 - 2] ...$$
$$y(0) = x[0]h[1] + x[1]h[0]$$

Therefore, an algorithm for setting up a 1D convolution would just be tracking the indices for $x$ and $h$, resulting in pseudocode that looks like

```
1. Generate the output array of len(input)
2. Iterate through the indices i for input to begin iterating through n
3. Iterate through the kernel j to perform the summation
4. Once the difference between i and j is less than 0, stop adding to the list
```

In [ ]:
@njit()
def conv_3d_loop_same(in_real, in_imag, kernel, out_real, out_imag):
    row, col, dep = in_real.shape
    krow, kcol, kdep = kernel.shape
    orow, ocol, odep = out_real.shape
    
    kcenterx = krow//2
    kcentery = kcol//2
    kcenterz = kdep//2

    for i in range(0, orow):
        for j in range(0, ocol):
            for k in range(0, odep):
                for m in range(0, krow):
                    for n in range(0, kcol):
                        for l in range(0, kdep):
                            ri = i + (m - kcenterx)
                            rj = j + (n - kcentery)
                            rk = k + (l - kcenterz)

                            if (ri >= 0) and (ri < row) and (rj >= 0) and (rj < col) and (rk >= 0) and (rk < dep):
                                out_real[i,j,k] += kernel[m,n,l]*in_real[ri,rj,rk]
                                out_imag[i,j,k] += kernel[m,n,l]*in_imag[ri,rj,rk]

@njit()
def conv_3d_loop_full(in_real, in_imag, kernel, out_real, out_imag, mode = 'fft'):
    row, col, dep = input.shape
    krow, kcol, kdep = kernel.shape
    orow, ocol, odep = out_real.shape
    if mode != 'fft':
        for i in range(0, orow):
            for j in range(0, ocol):
                for k in range(0, odep):
                    for m in range(0, krow):
                        for n in range(0, kcol):
                            for l in range(0, kdep):
                                ri = i - m
                                rj = j - n
                                rk = k - l

                                if (ri >= 0) and (ri < row) and (rj >= 0) and (rj < col) and (rk >= 0) and (rk < dep):
                                    out_real[i,j,k] += kernel[m,n,l]*in_real[ri,rj,rk]
                                    out_imag[i,j,k] += kernel[m,n,l]*in_imag[ri,rj,rk]
    else:
        # Function to perform 3D convolution using FFT
        signal = np.pad(in_real, [(0, orow-row), (0, ocol-col), (0, odep-dep)])
        kernel = np.pad(kernel, [(0, orow-krow), (0, ocol-kcol), (0, odep-kdep)])
    
        # Perform forward FFT on both signal and kernel
        signal_fft = fft.fftn(signal)
        kernel_fft = fft.fftn(kernel)
    
        # Element-wise multiplication in the frequency domain
        convolved_fft = signal_fft * kernel_fft
    
        # Inverse FFT to get back to the spatial domain
        convolved = fft.ifftn(convolved_fft)
        out_real = convolved.real
        out_imag = convolved.imag


def convolution_3D(input, kernel, mode = "full"):
    row, col, dep = input.shape
    krow, kcol, kdep = kernel.shape

    if mode == "same":
        ## Size of output ##
        newRows = row
        newCols = col
        newDeps = dep
        ## Size of output ##

        ## setup ##
        in_real = input.real
        in_imag = input.imag
        out_real = np.zeros((newRows, newCols, newDeps))
        out_imag = np.zeros((newRows, newCols, newDeps))
        ## setup ##
        
        ## convolution calculation ##
        conv_3d_loop_same(in_real, in_imag, kernel, out_real, out_imag)
        output = np.zeros((newRows, newCols, newDeps), dtype = complex)
        output.real = out_real
        output.imag = out_imag
        ## convolution calculation ##
    elif mode == "full":
        ## Size of output ##
        newRows = row + krow - 1
        newCols = col + kcol - 1
        newDeps = dep + kdep - 1
        ## Size of output ##

        ## setup ##
        in_real = input.real
        in_imag = input.imag
        out_real = np.zeros((newRows, newCols, newDeps))
        out_imag = np.zeros((newRows, newCols, newDeps))
        ## setup ##

        ## convolution calculation ##
        conv_3d_loop_full(in_real, in_imag, kernel, out_real, out_imag)
        output = np.zeros((newRows, newCols, newDeps), dtype = complex)
        output.real = out_real
        output.imag = out_imag
        ## convolution calculation ##
    else:
        raise AttributeError(f"{mode} is an invalid mode")
    
    return output


dims = [4, 4, 4]
slc = np.s_[:dims[0], :dims[1], :dims[2]]

density = np.ones(dims)
input = fft.fftn(density)/np.prod(dims)

filter = np.ones(dims)

loop_test = convolution_3D(input, filter, mode = "full")
scipy_test = scipy.signal.convolve(input, filter, mode = "full")
print("mode=full match:", np.unique(np.isclose(loop_test[slc] - scipy_test[slc], 0, atol = 1e-7))[0])

loop_test = convolution_3D(input, filter, mode = "same")
scipy_test = scipy.signal.convolve(input, filter, mode = "same")
print("mode=same match:", np.unique(np.isclose(loop_test - scipy_test, 0, atol = 1e-7))[0])

In [ ]:
L = 32

chi = 1.1
T = 0.5
kappa = 0.01

phi0 = -newton(fb, x0 = (0.5), args = (chi, T))
xi = np.sqrt((0.142*kappa)/((chi/2 - T)-0.31*T*(phi0**2)))
fit_func = lambda x, b:phi0*np.tanh((x - b)/xi)

# dims = [L, L]
# slc = np.s_[:L, :L]

dims = [L, L, L]
slc = np.s_[:dims[0], :dims[1], 0]
# slc = np.s_[:dims[0], 0, :dims[2]]
# slc = np.s_[0, :dims[1], :dims[2]]

idxs = np.indices(dims)
## Creating hydrovars parameters ##
rho = np.ones(dims)
phi = np.zeros(dims)
## Creating hydrovars parameters ##

## Creating inhomogenous system ##
# phi[idxs[0] >= L//2 + 1] = 0.502
# phi[idxs[0] <= L//2] = -0.502
phi = fit_func(idxs[0], (L - 1)/2)
## Creating inhomogenous system ##

## Convolving FFT with kernel ##
kernel = np.ones(dims)
rhok = fft.fftn(rho)/(L*L*L)
phik = fft.fftn(phi)/(L*L*L)
mode = "same"
# mode = "full"
rho_convolve = scipy.signal.convolve(rhok, kernel, mode = mode, method='fft')
phi_convolve = scipy.signal.convolve(phik, kernel, mode = mode, method='fft')
# rho_convolve = convolution_3D(rhok, kernel, mode = mode)
# phi_convolve = convolution_3D(phik, kernel, mode = mode)
## Convolving FFT with kernel ##

## Multiplying convolution with its adjoint ##
complex_adjoint = rho_convolve.conj()
rho_convolve = rho_convolve*complex_adjoint
complex_adjoint = phi_convolve.conj()
phi_convolve = phi_convolve*complex_adjoint
## Multiplying convolution with its adjoint ##

sz = 4
ar = 1
row = 2
col = 4
fig = plt.figure(1, figsize=(sz*col*ar, sz*row))
# numbers are in row-col-figure with figure number being row major

fig.add_subplot(241)
plt.imshow(rho[slc])
plt.colorbar()
plt.title(r"$\rho(r)$")

fig.add_subplot(245)
plt.imshow(phi[slc])
plt.colorbar()
plt.title(r"$\phi(r)$")

fig.add_subplot(242)
plt.imshow(rhok.real[slc])
plt.colorbar()
plt.title(r"$\rho(k)$")

fig.add_subplot(246)
plt.imshow(phik.real[slc])
plt.colorbar()
plt.title(r"$\phi(k)$")

fig.add_subplot(243)
plt.imshow(rho_convolve.real[:, 0, :], vmin = rho_convolve.real.min(), vmax = rho_convolve.real.max())
plt.colorbar()
plt.title(r"$real(\rho_k^{conv})$")

fig.add_subplot(247)
plt.imshow(phi_convolve.real[:, 0, :], vmin = phi_convolve.real.min(), vmax = phi_convolve.real.max())
plt.colorbar()
plt.title(r"$real(\phi_k^{conv})$")

fig.add_subplot(244)
plt.imshow(rho_convolve.imag[:, 0, :], vmin = rho_convolve.imag.min(), vmax = phi_convolve.imag.max())
plt.colorbar()
plt.title(r"$imag(\rho_k^{conv}$)")

fig.add_subplot(248)
plt.imshow(phi_convolve.imag[:, 0, :], vmin = phi_convolve.imag.min(), vmax = phi_convolve.imag.max())
plt.colorbar()
plt.title(r"$imag(\phi_k^{conv}$)")

fig.tight_layout()
print(rho_convolve.shape)

In [53]:
# Function to perform 3D convolution using FFT
def fft_convolve3d_same(signal, kernel):
    # Pad the kernel to the size of the signal to prevent circular convolution
    padded_kernel = np.zeros_like(signal)
    kernel_shape = kernel.shape
    padded_kernel[:kernel_shape[0], :kernel_shape[1], :kernel_shape[2]] = kernel
    
    # Perform forward FFT on both signal and kernel
    signal_fft = fft.fftn(signal)
    kernel_fft = fft.fftn(padded_kernel)
    
    # Element-wise multiplication in the frequency domain
    convolved_fft = signal_fft*kernel_fft
    
    # Inverse FFT to get back to the spatial domain
    convolved = fft.ifftn(convolved_fft)
    
    return convolved

# Function to perform 3D convolution using FFT
def fft_convolve3d_full(signal, kernel):
    # Pad the kernel and signal to the required output shape
    row, col, dep = signal.shape
    krow, kcol, kdep = kernel.shape

    orow = row + krow - 1
    ocol = col + kcol - 1
    odep = dep + kdep - 1

    signal = np.pad(signal, [(0, orow-row), (0, ocol-col), (0, odep-dep)])
    kernel = np.pad(kernel, [(0, orow-krow), (0, ocol-kcol), (0, odep-kdep)])
    
    # Perform forward FFT on both signal and kernel
    signal_fft = fft.fftn(signal)
    kernel_fft = fft.fftn(kernel)
    
    # Element-wise multiplication in the frequency domain
    convolved_fft = signal_fft * kernel_fft
    
    # Inverse FFT to get back to the spatial domain
    convolved = fft.ifftn(convolved_fft)
    
    return convolved

In [ ]:
input = phik
mode = "full"

if mode == 'same':
    out_pad = fft_convolve3d_same(input, kernel)
elif mode == 'full':
    out_pad = fft_convolve3d_full(input, kernel)

conv_test = out_pad*out_pad.conj()

sz = 4
col = 2
row = 1
fig = plt.figure(1, figsize=(sz*col*ar, sz*row))

fig.add_subplot(221)
plt.imshow(out_pad[slc].real)
plt.colorbar()

fig.add_subplot(223)
plt.imshow(out_pad[slc].imag)
plt.colorbar()

fig.add_subplot(222)
plt.imshow(conv_test[slc].real)
plt.colorbar()

fig.add_subplot(224)
plt.imshow(conv_test[slc].imag)
plt.colorbar()

In [ ]:
for i in range(L//2):
    print(f"kx:{i} ", rho_convolve[i, 0, 0])

In [ ]:
for i in range(L//2):
    print(f"kx:{i} ", phi_convolve[i, 0, 0])

### Investigating where the free energy remains bounded

In [ ]:
# %%capture
@njit
def func_mu_rho(rho0, phi0, T, chi, k2, kappa):
    numerator = (-2.*T*np.power(rho0, 4) - chi*np.power(phi0, 4) + chi*np.power(phi0*rho0, 2))
    denominator = (2.*np.power(rho0, 3)*(np.power(phi0, 2) - np.power(rho0, 2)))
    out = numerator/denominator + k2*kappa
    return out

@njit 
def func_mu_phi(rho0, phi0, T, chi, k2, kappa):
    numerator = (-2.*T*np.power(rho0, 2) - chi*np.power(phi0, 2) + chi*np.power(rho0, 2))
    denominator = (2.*rho0*(np.power(phi0, 2) - np.power(rho0, 2)))
    # print(numerator, denominator)
    out = numerator/denominator + k2*kappa
    return out

L = 64
freqs = fft.fftshift(fft.fftfreq(L))
kx, ky, kz = np.meshgrid(*tuple([2*np.pi*freqs for L in [L, L, L]]), indexing='ij')
k2 = lattice_fourier_laplacian(kx, ky, kz)

k2 = np.unique(np.round(k2, 1))
phi_s = np.linspace(-0.90, 0.90, k2.size)
rho_s = np.linspace(0.995, 1.005, k2.size)

kappa_set = 0.5
chi_set = 0.8
T_set = 0.35

K2, PHIS = np.meshgrid(*[k2, phi_s])
MUPHI = np.zeros_like(K2)
K2, RHOS = np.meshgrid(*[k2, rho_s])
MURHO = np.zeros_like(K2)

for i in range(k2.size):
    for j in range(k2.size):
        MUPHI[i,j] = func_mu_phi(1, PHIS[i, j], T_set, chi_set, K2[i, j], kappa_set)
        MURHO[i,j] = func_mu_rho(1, RHOS[i, j], T_set, chi_set, K2[i, j], kappa_set)

fig, axs = plt.subplots(1, 2, figsize = (8, 4))

ax = axs[0]
im = ax.pcolor(K2,PHIS,MUPHI, vmin = -0.01, vmax = 0.01)
plt.colorbar(im, ax = ax, label = r"$\mu_{\phi}$", orientation = 'horizontal')
ax.set_xlabel("k2")
ax.set_ylabel(r"$\phi$")

ax.set_title(f"T = {T_set}, $\chi$ = {chi_set}")
fig.tight_layout()

ax = axs[1]
im = ax.pcolor(K2,RHOS,MURHO, vmin = -0.01, vmax = 0.01)
plt.colorbar(im, ax = ax, label = r"$\mu_{\rho}$", orientation = 'horizontal')
ax.set_xlabel("k2")
ax.set_ylabel(r"$\rho$")

ax.set_title(f"T = {T_set}, $\chi$ = {chi_set}")
fig.tight_layout()

### Calculating the general interface profile empirically

In [ ]:
fig, axs = plt.subplots(1, 3, figsize = (9, 3))

kappa_s = [0.01, 0.02, 0.03]
b_k = [-2.312, -1.5426, -1.1367]

chi_s = [1.1, 1.2, 1.3]
b_chi = [-2.312, -3.0679, -3.4158]

T_s = [0.48, 0.5, 0.52]
b_T = [-2.6976, -2.314, -1.7270]

ax = axs[0]
x = np.array(kappa_s)
y = np.array(b_k)
x = np.log(x)
y = np.log(np.abs(y))

ax.plot(x, y, 'bx--', label = "raw")

fit_func = lambda x, b, c: b*x + c
popt, pcov = curve_fit(fit_func, x, y)
print(popt)

xfit = np.linspace(x.min(), x.max(), 101)
yfit = fit_func(xfit, *popt)
ax.plot(xfit, yfit, 'r-', label = "fit")

ax.legend()
ax.set_xlabel(r"$\kappa$")
ax.set_ylabel(r"$b$")

ax = axs[1]
x = np.array(chi_s)
y = np.array(b_chi)
x = np.log(x)
y = np.log(np.abs(y))

ax.plot(x, y, 'bx--', label = "raw")

fit_func = lambda x, b, c: b*x + c
popt, pcov = curve_fit(fit_func, x, y)
print(popt)

xfit = np.linspace(x.min(), x.max(), 101)
yfit = fit_func(xfit, *popt)
ax.plot(xfit, yfit, 'r-', label = "fit")

ax.legend()
ax.set_xlabel(r"$\chi$")
ax.set_ylabel(r"$b$")

ax = axs[2]
x = np.array(T_s)
y = np.array(b_T)
x = np.log(x)
y = np.log(np.abs(y))

ax.plot(x, y, 'bx--', label = "raw")

fit_func = lambda x, b, c: b*x + c
popt, pcov = curve_fit(fit_func, x, y)
print(popt)

xfit = np.linspace(x.min(), x.max(), 101)
yfit = fit_func(xfit, *popt)
ax.plot(xfit, yfit, 'r-', label = "fit")

ax.legend()
ax.set_xlabel(r"$T$")
ax.set_ylabel(r"$b$")

fig.tight_layout()

We will also try to calculate the theoretical interfacial profile from the underlying math. The total free energy is defined as 

$$F(\rho, \phi, T, \chi) = f_0(\rho, \phi, T, \chi) + \frac{\kappa}{2}(\nabla \rho)^2 + \frac{\kappa}{2}(\nabla \phi)^2$$

Where $\rho$ and $\phi$ represent the sum and difference of the concentrations of fluids at each site, $T$ is the temperature of the system and $\chi$ is the thermodynamic parameter of the system. 

$$f_0(\rho, \phi, T, \chi) = \frac{\chi}{4}\rho(1 - (\frac{\phi}{\rho})^2) - \rho T + \frac{T}{2}(\rho + \phi)\ln{\frac{\rho + \phi}{2}} + \frac{T}{2}(\rho - \phi)\ln{\frac{\rho - \phi}{2}}$$

To obtain the interfacial profile, the chemical potential of $\phi$ will be used, defined as

$$\mu_{\phi} = -\frac{\chi}{2}\frac{\phi}{\rho} + \frac{T}{2}\ln{\frac{ 1 + \frac{\phi}{\rho} }{1 - \frac{\phi}{\rho}}} - \kappa \nabla^2 \phi$$

We will assume that $\rho = 1$, simplifying the above to

$$\mu_{\phi} = -\frac{\chi}{2}\phi + \frac{T}{2}\ln{\frac{ 1 + \phi }{1 - \phi}} - \kappa \nabla^2 \phi$$

To obtain the profile of the interface, we will assume that the system is at thermodynamic equilibrium, meaning that $\mu_{\phi} = 0$. We now have a non-linear ODE due to the $\phi$ terms in the $\ln$ term. We can then split the $\ln$ into two terms, then taylor expand the terms.

\begin{equation}
\ln(1 + \phi) =  \phi - \frac{\phi^2}{2} + \frac{\phi^3}{3} + O(x^4)
\end{equation}

\begin{equation}
\ln(1 - \phi) = -\phi - \frac{\phi^2}{2} - \frac{\phi^3}{3} + O(x^4)
\end{equation}

\begin{equation}
\ln(1 + \phi) - \ln(1 - \phi) = 2\phi + \frac{2\phi^3}{3}
\end{equation}

After the taylor expansion and $\ln$ term substitution, we get

$$\mu_{\phi} = -\frac{\chi}{2}\phi + T\phi + T\frac{\phi^3}{3} - \kappa \nabla^2 \phi$$

We assume that the interface profile has a profile defined as $\phi = \phi_0 \tanh{\frac{x}{\xi}}$. After substitution and simplification to solve for the value of $\xi$, we identify that the profile can be defined with the expression,

$$\phi = \phi_0 \tanh{\frac{x}{\xi}}$$

$$\xi = \sqrt{\frac{0.142\kappa}{(\frac{\chi}{2} - T) - 0.31T\phi_0^2}}$$

The value of $\phi_0$ is defined as the positive root of the bulk free energy, solved using the newton raphson numerical method. With this expression, an analytical solution for the surface tension can also be found by integrating the excess free energy from $-\infty$ to $\infty$

$$\sigma = \int_{\infty}^{\infty} \frac{\kappa}{2}(\nabla \phi)^2$$

After substituting the expression for $\phi$ and using sympy and wolfram alpha to evaulate the integral, the final analytical expression for the surface tension is found to be

$$\sigma = 2 \phi_0^2[1.76(\chi - 2T) - 1.092T\phi_0^2]$$

In [ ]:
import sympy as sp 

x = sp.symbols('x')
a = sp.symbols('alpha')
Cm = (sp.Rational(1) + sp.tanh(x/(2*a)))/sp.Rational(2)
dCm = sp.Derivative(Cm, x, 1).doit()

F = (Cm**2)*(1 - Cm)**2 + (a**2)*(dCm**2)

sp.integrate(F, x)

In [111]:
chi, T, phi, phi0, kappa, x = sp.symbols("chi, T, phi, phi0, kappa, x")

# b = (chi**2.356)*(T**5.559)/(kappa**0.640)
b = sp.sqrt(kappa*(chi/2 - T)/T)
phi = phi0*sp.tanh(b*x)
dphi = sp.Derivative(phi, x, 1).doit()

f0 = chi/4*(1 - phi**2) - T + T/2*(1 + phi)*sp.ln((1 + phi)/2) + T/2*(1 - phi)*sp.ln((1 - phi)/2)

expr = sp.integrate(kappa/2*(dphi**2), x).simplify()
# F = 

In [ ]:
chi = 1.1
T = 0.5
kappa = 0.03

def fb(phi, l, T, rho = 1):
    return -(l/2)*(phi/rho) + (T/2)*np.log((1 + phi/rho)/(1 - phi/rho))
phi0 = newton(fb, x0 = (0.55), args = (chi, T))

phi_theory = np.sqrt(3*(chi/2 - T)/T)
print(f"phi_real:{phi0:.3f} phi_theory:{phi_theory:.3f} Ratio:{phi_theory/phi0:.3f}")

In [ ]:
L = 5

x = np.linspace(-L, L, 2*L + 1)

b_model = (np.power(chi, 2.350)/(pow(kappa, 0.640)*np.power(T, 5.559)*489.435))
profile = phi0*np.tanh(b_model*x)
plt.plot(x, profile, "ro", label = "old fit", markerfacecolor = "None")

xi = np.sqrt((0.142*kappa)/((chi/2 - T)-0.31*T*(phi0**2)))
y = phi0*np.tanh(x/xi)

plt.plot(x, y, 'bs', label = "new fit", markerfacecolor = "None")
plt.legend()

In [51]:
import sympy as sp 

phi0, chi, kappa, T, x = sp.symbols("phi_0 chi kappa T x")

# xi = sp.sqrt((sp.Rational(0.142)*kappa)/((chi/2 - T) - sp.Rational(0.31)*T*(phi0**2)))
xi = sp.sqrt((0.142*kappa)/((chi/2 - T) - 0.31*T*(phi0**2)))
xi = xi.simplify()
phi = phi0*sp.tanh(x/xi)
dphi = sp.Derivative(phi, x, 1).doit().simplify()

excess_energy = kappa/2*(dphi**2)
bulk_energy = chi/4*(1 - phi**2) - T + T/2*(1+phi)*sp.ln((1+phi)/2) + T/2*(1-phi)*sp.ln((1-phi)/2)

total_energy = excess_energy + bulk_energy

c = 1.1
temp = 0.5
k = 0.03
# total_energy = total_energy.subs({chi:c, kappa:k, T:temp, phi0:newton(fb, x0 = (0.55), args = (c, temp))}).simplify()

unsub_expr = sp.integrate(excess_energy, (x, -sp.oo, sp.oo)).simplify()

excess_energy = excess_energy.subs({chi:c, kappa:k, T:temp, phi0:newton(fb, x0 = (0.55), args = (c, temp))}).simplify()
sub_expr = sp.integrate(excess_energy, (x, -sp.oo, sp.oo)).simplify()

In [ ]:
unsub_expr#.subs({chi:c, kappa:k, T:temp, phi0:newton(fb, x0 = (0.55), args = (c, temp))}).simplify()

In [ ]:
import sympy as sp

phi0, chi, kappa, T, x = sp.symbols("phi_0 chi kappa T x")
xi = sp.sqrt(kappa/(chi/2 - T))
phi = phi0*sp.tanh(x/xi)
dphi = sp.Derivative(phi, x, 1).doit().simplify()

excess_energy = kappa/2*(dphi**2)
bulk_energy = chi/4*(1 - phi**2) - T + T/2*(1+phi)*sp.ln((1+phi)/2) + T/2*(1-phi)*sp.ln((1-phi)/2)

total_energy = excess_energy + bulk_energy

c = 0.5
temp = 0.2
k = 0.03
phi0_real = sp.sqrt(3*(c/2 - temp)/temp)
xi_real = sp.sqrt(k/(c/2 - temp))
props = {chi:c, kappa:k, T:temp, phi0:phi0_real}

# total_energy = excess_energy.subs({chi:c, kappa:k, T:temp, phi0:phi0_real}).simplify()
# sub_expr = sp.integrate(total_energy, (x, -sp.oo, sp.oo)).simplify()

total_energy = total_energy.simplify()
unsub_expr = sp.integrate(total_energy, (x, -sp.oo, sp.oo)).simplify()
# sub_expr

In [ ]:
unsub_expr.subs(props)

### Finite size effects in the fluctuations of the interface height

In [ ]:
import scipy.stats as stats

# fig, axs = plt.subplots(1, 2, figsize = (9, 3))
f, (ax1, ax2) = plt.subplots(1, 2,  figsize = (10, 5), gridspec_kw={'width_ratios': [3, 2]})
# color = [""]
chi = 1.1
T = 0.5
z = 64
h = np.zeros(z)
ls = []
paths = sorted(glob.glob(f"./2D_int/L_{z}/*.h5"))[-2:]
for path in paths:
    phi = extract_data(path, [16, 6, z], begin = 1, end = 2, nVars = 38)
    temph = interface_height(phi[0], chi, T)
    h += np.mean(temph, axis = 0)
    ls += list(temph)
h = h/len(paths)
x = np.linspace(0, z - 1, z)
ax1.plot(x/x.max(), h - h.mean(), label = f"z = {z}")
ls = np.array(ls)
ls -= ls.mean()
ax2.hist(ls.flatten(), density = True, label = f"z = {z}", alpha = 0.5, bins = 20)
# mu = 0
# sigma = np.std(h)
# x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
# ax2.plot(x, stats.norm.pdf(x, mu, sigma), label = f"z = {z}")

z = 128
h = np.zeros(z)
ls = []
paths = sorted(glob.glob(f"./2D_int/L_{z}/*.h5"))[-2:]
for path in paths:
    phi = extract_data(path, [16, 6, z], begin = 1, end = 2, nVars = 38)
    temph = interface_height(phi[0], chi, T)
    h += np.mean(temph, axis = 0)
    ls += list(temph)
h = h/len(paths)
x = np.linspace(0, z - 1, z)
ax1.plot(x/x.max(), h - h.mean(), label = f"z = {z}")
ls = np.array(ls)
ls -= ls.mean()
ax2.hist(ls.flatten(), density = True, label = f"z = {z}", alpha = 0.5, bins = 20)

z = 256
h = np.zeros(z)
ls = []
paths = sorted(glob.glob(f"./2D_int/L_{z}/*.h5"))[-2:]
for path in paths:
    phi = extract_data(path, [16, 6, z], begin = 1, end = 2, nVars = 38)
    temph = interface_height(phi[0], chi, T)
    h += np.mean(temph, axis = 0)
    ls += list(temph)
h = h/len(paths)
x = np.linspace(0, z - 1, z)
ax1.plot(x/x.max(), h - h.mean(), label = f"z = {z}")
ls = np.array(ls)
ls -= ls.mean()
ax2.hist(ls.flatten(), density = True, label = f"z = {z}", alpha = 0.5, bins = 20)

z = 512
h = np.zeros(z)
ls = []
paths = sorted(glob.glob(f"./2D_int/L_{z}/*.h5"))[-2:]
for path in paths:
    phi = extract_data(path, [16, 6, z], begin = 1, end = 2, nVars = 38)
    temph = interface_height(phi[0], chi, T)
    h += np.mean(temph, axis = 0)
    ls += list(temph)
h = h/len(paths)
x = np.linspace(0, z - 1, z)
ax1.plot(x/x.max(), h - h.mean(), label = f"z = {z}")
ls = np.array(ls)
ls -= ls.mean()
ax2.hist(ls.flatten(), density = True, label = f"z = {z}", alpha = 0.5, bins = 20)

ax1.legend(ncols = 2)
ax2.legend()
fig.tight_layout()

### Calculations

In [ ]:
boxDim = [64, 2, 64]

output_file = "hydro_plt"
path = sorted(glob.glob(f"2D_int/{output_file}*.h5"))[idx]
profile = extract_data(path, boxDim, begin = 0, end = 2, nVars = 38)

# plt.imshow(profile[1, :, 0, :])

phi0 = -newton(fb, x0 = (0.6), args = (chi, T))
xi = np.sqrt((0.142*kappa)/((chi/2 - T)-0.31*T*(phi0**2)))
xi = np.sqrt(kappa/(chi/2 - T))
# fit_func = lambda x, b, c: newton(fb, x0 = (0.5), args = (chi, T))*np.tanh(b*(x - c))
shift = 1/np.sqrt(2)
fit_func = lambda x, b:phi0*np.tanh(shift*(x - b)/xi)

# slc = slice(64//4, 3*64//4)
slc = np.s_[1, 64//4:3*64//4, 0, 2]
xraw = np.arange(64//4, 3*64//4, 1)
interface_slc = profile[slc]
plt.plot(xraw, interface_slc, 'rx')

popt,pcov = curve_fit(fit_func, xraw, interface_slc, p0 = [64//2 - 0.5])
xfit = np.linspace(64//4, 3*64//4, 1001)
yfit = fit_func(xfit, *popt)
plt.plot(xfit, yfit, '-')
popt

In [9]:
nx = 128
ny = 2
nz = 2048

boxDim = np.array([nx, ny, nz])

kappa = 0.03
kbt = 1e-7
u0 = 0
cs2 = 1/3
gamma = 1.0

chi = 0.5
T = 0.2

idx = -1
# noise_type = "spatially_independent"
noise_type = "spatially_dependent"
savedir = f"./validation/interface/{noise_type}"
# savedir = f"./validation/interface/spatially_dependent/"
output_file = "hydro_plt"

In [ ]:
boxDim = np.array([nx, ny, nz])

# profile = extract_data(savedir + sorted(glob.glob("./*.h5"))[-1], [nx, ny, nz], begin = 1, end = 2, nVars = 38)
path = sorted(glob.glob(f"{savedir}/{output_file}*.h5"))[idx]
profile = extract_data(path, boxDim, begin = 0, end = 2, nVars = 38)

# fit_func = lambda x, b, c: newton(fb, x0 = (0.6), args = (chi, T))*np.tanh(b*(x - c))

phi0 = -newton(fb, x0 = (0.6), args = (chi, T))
xi = np.sqrt((0.142*kappa)/((chi/2 - T)-0.31*T*(phi0**2)))
# fit_func = lambda x, b, c: newton(fb, x0 = (0.5), args = (chi, T))*np.tanh(b*(x - c))
fit_func = lambda x, b:phi0*np.tanh(0.76*(x - b)/xi)

fig, axs = plt.subplots(2, 2, figsize = (8, 8))
axs = axs.flatten()

ax = axs[0]
im = ax.imshow(profile[0, :, : , nz//2])
ax.set_xlabel("y")
ax.set_ylabel("x")
plt.colorbar(im, ax = ax, label = r"$\rho$")

ax = axs[1]
x = np.arange(nx//4, 3*nx//4, 1)
im = ax.plot(x, profile[0, nx//4:3*nx//4, ny//2 , nz//2], 'rx', label = "profile")
ax.plot(x, np.ones(x.size), 'ko', label = "reference", markerfacecolor="None")

ax.set_xlabel("x")
ax.set_ylabel(r"$\rho$")
ax.legend(ncol = 1, fontsize = 'small')

ax = axs[2]
im = ax.imshow(profile[1, :, : , nz//2])
ax.set_xlabel("y")
ax.set_ylabel("x")
plt.colorbar(im, ax = ax, label = r"$\phi$")

ax = axs[3]
x = np.arange(nx//4, 3*nx//4, 1)
im = ax.plot(x, profile[1, nx//4:3*nx//4, ny//2 , nz//2], 'rx', label = "profile")

# x = np.linspace(0, nx - 1, nx)
# b = -(chi**2.350)/(kappa**0.640 * T**5.559*489.435)
# y = newton(fb, x0 = (0.6), args = (chi, T))*np.tanh(b*(x - (nx - 1)/2))
y = fit_func(x, (nx - 1)/2)
ax.plot(x, y, 'ko', label = "referece", markerfacecolor="None")

ax.set_xlabel("x")
ax.set_ylabel(r"$\phi$")
ax.legend(ncol = 1, fontsize = 'small')

t = int(path.split("/")[-1].split(".")[0].split("_")[-1])
fig.suptitle(f"$\kappa$ = {kappa}, $\chi = {chi}, T = {T}, t = {t}$")
fig.tight_layout()

In [ ]:
method = "direct"
fft_mode = 'forward'

# method = "profile_fit"
# fft_mode = 'ortho'

heights_k = np.zeros((nz), dtype = np.complex128)
h5_paths = sorted(glob.glob(f"{savedir}/{output_file}*.h5"))[1:]

for path in h5_paths:
    phi = extract_data(path, boxDim, begin = 1, end = 2, nVars = 38)[0]
    
    h = interface_height(phi, chi, T, kappa, method = method)
    h = np.mean(h, axis = 0)

    h_k = fft.fft(h, norm = fft_mode)

    heights_k += h_k*h_k.conjugate()
    t = int(path.split("/")[-1].split(".")[0].split("_")[-1])
    print(f"Timestep {t} processed")

heights_k = np.abs(heights_k)
heights_k /= len(h5_paths)

In [ ]:
## EXPERIMENTAL RESULTS ##
L = nz
k1 = fft.fftfreq(L)*2*np.pi
S1 = heights_k.copy()

slc = slice(1, L//2)
xraw = k1[slc]
yraw = S1[slc]

binsize = L//4
kmin = 2*np.pi/binsize
bins = np.arange(binsize//2+1)*kmin # kmax+1 for bin_edges: len(bins)=len(hist)+1
shells = np.histogram(k1[slc], bins, weights=S1[slc])[0]
counts = np.histogram(k1[slc], bins)[0]

xbin = (bins[:-1]+bins[1:])/2
ybin = shells/counts
## EXPERIMENTAL RESULTS ##

## THEORETICAL RESULTS ##
# phi0 = newton(fb, x0 = (0.6), args = (chi, T))
# sigma = 2*(phi0**2)*(1.76*(chi - 2*T) - 1.092*T*(phi0**2))
# sigma = 5e-4
sigma = 0.0172

freqs_theory = np.linspace(xraw[0], xraw[-1], 1001)
interface_fluct_theory = kbt/(sigma*np.power(freqs_theory, 2))
## THEORETICAL RESULTS ##

sz = 4
ar = 1.25
fig, ax = plt.subplots(1, 1, figsize = (sz*ar, sz))

# shift = ny/(nx*ny)
# shift = 1/np.sqrt(nx*nz*ny**2)
shift = np.sqrt(ny)
# shift = ny
# shift = 1
# ax.loglog(xraw, yraw, "ks", label = "Simulation raw", ms = 5, markerfacecolor = "None")
ax.loglog(xbin, ybin*shift, "bo", label = "Simulation", ms = 3)
ax.loglog(freqs_theory, interface_fluct_theory, 'r-', lw = 2, label = "Theory")

ax.set_xlim(left = 1e-2)
ax.set_xlabel(r"$k$", fontsize = 15)
ax.set_ylabel(r"$\langle |h(k)|^2 \rangle$", fontsize = 15)
ax.tick_params(axis='both', which='major', labelsize=12)

ax.legend(fontsize = 12, loc = 'lower left')
# ax.set_title(f"L = {nz}")
ax.set_title(f"{method}")
fig.tight_layout()

fig.savefig(f"figures/interface_height_flucuations-{noise_type}.png", dpi = 300)

## Droplet fluctuations

Sources
1. [Simulation of FUS Protein Condensates with an adapted coarse grained model](https://pubs.acs.org/doi/10.1021/acs.jctc.0c01064)
2. [Effect of nanoparticles and surfactants on droplets in shear flows](https://doi.org/10.1039/C2SM25209K)

### Young Laplace fit

In [20]:
nx = 64
ny = 64
nz = 64

boxDim = np.array([nx, ny, nz])

kappa = 0.03
kbt = 1e-7
u0 = 0
cs2 = 1/3
gamma = 1.0

chi = 0.5
T = 0.2

idx = -1
savedir = f"./validation/droplet_fluctuations/young_laplace"
R_s = ["0.2", "0.25", "0.3", "0.35"]
output_file = "hydro_plt"

In [21]:
cutoff = 0

all_radii = []
all_times = []
all_press = []

for R in R_s:
    paths = sorted(glob.glob(f"{savedir}/R_{R}/{output_file}*.h5"))

    curr_radii = np.zeros(len(paths))    
    curr_times = np.zeros(len(paths))
    curr_press = np.zeros(len(paths))
    for i, path in enumerate(paths):
        t = int(path.split("_")[-1].split(".")[0])    
        curr_times[i] = t

        profile = extract_data(path, boxDim, begin = 1, end = 2, nVars = 38)
        curr_radii[i] = droplet_radius(profile[0])

        profile = extract_data(path, boxDim, begin = 0, end = 1, nVars = 38)
        curr_press[i] = pressure_jump(profile[0]*T)
        
    
    all_radii.append(curr_radii)
    all_times.append(curr_times)
    all_press.append(curr_press)

In [ ]:
sz = 4
ar = 1.25
fig, axs = plt.subplots(1, 2, figsize = (sz*ar*2, sz))
axs2 = [ax.twinx() for ax in axs]

cols = ['r', 'b', 'k', 'g']
markers = ['s', 'o', '^', '*']
linestyles = ['-', '--', ':', 'dashdot']

for i, R in enumerate(R_s):
    x = all_times[i]
    y = all_radii[i]
    y2 = all_press[i]
    if len(x) > 1:
        plotx = x[1:]
        ploty = y[1:]
        ploty2 = y2[1:]

        ax = axs[0]
        ax.plot(plotx, ploty, label = f'R = {R}', marker = markers[i], linestyle = "None", color = cols[i], markerfacecolor = "None", ms = 8)

        ax2 = axs2[0]
        ax2.plot(plotx, ploty2, label = f'R = {R}', marker = "None", linestyle = linestyles[i], color = cols[i], markerfacecolor = "None", ms = 8)

        ax = axs[1]
        ploty = (ploty - ploty.min())/(ploty.max() - ploty.min())
        ax.plot(plotx, ploty, label = f'R = {R}', marker = markers[i], linestyle = "None", color = cols[i], markerfacecolor = "None", ms = 8)

        ax2 = axs2[1]
        ploty = (ploty2 - ploty2.min())/(ploty2.max() - ploty2.min())
        ax2.plot(plotx, ploty, label = f'R = {R}', marker = "None", linestyle = linestyles[i], color = cols[i], markerfacecolor = "None", ms = 8)
    
ax = axs[0]

ax.set_xlabel("Timesteps")
ax.set_ylabel(r"$R$")
ax.set_xscale("log")
# ax.legend(loc = 'center right')

ax2 = axs2[0]
ax2.set_ylabel(r"$\Delta P$")

ax = axs[1]
ax.set_xlabel("Timesteps")
ax.set_ylabel(r"$\frac{R - R_{min}}{R_{max} - R_{min}}$")
ax.set_xscale("log")
# ax.legend(loc = 'center right')
ax.legend(loc='center left', bbox_to_anchor=(1.2, 0.5))

ax2 = axs2[1]
ax2.set_ylabel(r"$\frac{\Delta P - \Delta P_{min}}{\Delta P_{max} - \Delta P_{min}}$")


fig.suptitle("Droplet radius evolution over time")
fig.tight_layout()

In [ ]:
young_laplace = lambda x, sigma: sigma*x

xraw = []
yraw = []
for i in range(len(R_s)):
    ls = all_radii[i]
    if len(ls) > 1:
        xraw.append(2/all_radii[i][-1])
        yraw.append(all_press[i][-1])

sz = 4
ar = 1.25
fig, ax = plt.subplots(1, 1, figsize = (sz*ar, sz))

ax.plot(xraw, yraw, 'bo', ms = 8, linestyle = "None", label = "Raw data", markerfacecolor = "None")

popt, pcov = curve_fit(young_laplace, xraw, yraw, p0 = [0.017])
xfit = np.linspace(min(xraw), max(xraw), 11)
yfit = young_laplace(xfit, *popt)
ax.plot(xfit, yfit, 'r-', lw = 1, label = "Fit data")

ax.text(np.mean(xraw[:2]), np.mean(yraw[:2])-(yfit[-1] - yfit[-2]), r"$\sigma$="+f"{popt[0]:.4f}", fontsize = 12)
ax.set_ylabel(r"$\Delta P$", fontsize = 15)
ax.set_xlabel(r"$\frac{2}{R_d}$", fontsize = 15)
ax.set_title(f"$\chi$={chi}, $T$={T}, $\kappa$={kappa}", fontsize = 18)
ax.legend(fontsize = 12)
fig.savefig("figures/young_laplace_sigma.png", dpi = 300)

### Fluctuations of droplet shape

In [24]:
nx = 64
ny = 64
nz = 64

boxDim = np.array([nx, ny, nz])

kappa = 0.03
kbt = 1e-7
u0 = 0
cs2 = 1/3
gamma = 1.0

chi = 0.5
T = 0.2

idx = -1

# noise_type = "spatially_dependent"
noise_type = "spatially_independent"
savedir = f"./validation/droplet_fluctuations/{noise_type}"
output_file = "hydro_plt"

In [ ]:
boxDim = np.array([nx, ny, nz])

path = sorted(glob.glob(f"{savedir}/{output_file}*.h5"))[idx]
profile = extract_data(path, boxDim, begin = 0, end = 2, nVars = 38)

fig, axs = plt.subplots(2, 2, figsize = (8, 8))
axs = axs.flatten()

ax = axs[0]
im = ax.imshow(profile[0, :, : , nz//2])
ax.set_xlabel("y")
ax.set_ylabel("x")
plt.colorbar(im, ax = ax, label = r"$\rho$")

ax = axs[1]
x = np.arange(0, nx, 1)
im = ax.plot(x, profile[0, :, ny//2 , nz//2], 'rx', label = "profile")
ax.set_xlabel("x")
ax.set_ylabel(r"$\rho$")
ax.legend(ncol = 1, fontsize = 'small')

ax = axs[2]
im = ax.imshow(profile[1, :, : , nz//2])
ax.set_xlabel("y")
ax.set_ylabel("x")
plt.colorbar(im, ax = ax, label = r"$\phi$")

ax = axs[3]
x = np.arange(0, nx, 1)
im = ax.plot(x, profile[1, :, ny//2 , nz//2], 'rx', label = "profile")
ax.set_xlabel("x")
ax.set_ylabel(r"$\phi$")
ax.legend(ncol = 1, fontsize = 'small')

t = int(path.split("/")[-1].split(".")[0].split("_")[-1])
fig.suptitle(f"$\kappa$ = {kappa}, $\chi = {chi}, T = {T}, t = {t}$")
fig.tight_layout()

In [ ]:
rho = phi = profile[0]
phi = profile[1]

print(f"center phi:{phi[nx//2, ny//2, nz//2]}, edge:phi:{phi[0, 0, 0]}")

L, L, L = phi.shape
dims = [L, L, L]
slc = np.s_[:L, :L, L//2]

## Convolving FFT with kernel ##
kernel = np.ones(dims)
rhok = fft.fftn(rho)
phik = fft.fftn(phi)
mode = "full"
# mode = "same"
rho_convolve = scipy.signal.convolve(rhok, kernel, mode = mode)/np.prod(dims)
phi_convolve = scipy.signal.convolve(phik, kernel, mode = mode)/np.prod(dims)
## Convolving FFT with kernel ##

## Multiplying convolution with its adjoint ##
complex_adjoint = rho_convolve.conj()
rho_convolve = rho_convolve*complex_adjoint
complex_adjoint = phi_convolve.conj()
phi_convolve = phi_convolve*complex_adjoint
## Multiplying convolution with its adjoint ##

sz = 4
ar = 1
row = 2
col = 4
fig = plt.figure(1, figsize=(sz*col*ar, sz*row))
# numbers are in row-col-figure with figure number being row major

fig.add_subplot(241)
plt.imshow(rho[slc])
plt.colorbar()
plt.title(r"$\rho(r)$")

fig.add_subplot(245)
plt.imshow(phi[slc])
plt.colorbar()
plt.title(r"$\phi(r)$")

fig.add_subplot(242)
plt.imshow(rhok.real[slc])
plt.colorbar()
plt.title(r"$\rho(k)$")

fig.add_subplot(246)
plt.imshow(phik.real[slc])
plt.colorbar()
plt.title(r"$\phi(k)$")

fig.add_subplot(243)
plt.imshow(rho_convolve[slc].real, vmin = 1, vmax = 1)
plt.colorbar()
plt.title(r"$real(\rho_k^{conv})$")

fig.add_subplot(247)
plt.imshow(phi_convolve[slc].real, vmin = phi_convolve.real[slc].min(), vmax = phi_convolve.real[slc].max())
plt.colorbar()
plt.title(r"$real(\phi_k^{conv})$")

fig.add_subplot(244)
plt.imshow(rho_convolve[slc].imag, vmin = 0, vmax = 0)
plt.colorbar()
plt.title(r"$imag(\rho_k^{conv}$)")

fig.add_subplot(248)
plt.imshow(phi_convolve[slc].imag, vmin = 0, vmax = 0)
plt.colorbar()
plt.title(r"$imag(\phi_k^{conv}$)")

fig.tight_layout()
print(rho_convolve.shape)

In [ ]:
cutoff = 0
paths = sorted(glob.glob(f"{savedir}/{output_file}*.h5"))[:51]
 
R_s = np.zeros(len(paths))
times = np.zeros(len(paths))
radiis = np.zeros((len(paths), 3))

for i, path in enumerate(paths):
    t = int(path.split("_")[-1].split(".")[0])    
    times[i] = t
    
    profile = extract_data(path, boxDim, begin = 1, end = 2, nVars = 38)
    R_s[i] = droplet_radius(profile[0])

    field = profile[0].copy()
    
    field = np.where(field > cutoff, 0, field)
    cm = center_of_mass(field)
    I = inertia_tensor(cm,field)
    m = droplet_mass(field)
    radiis[i] = radii_pca(eigenvalues(I), m)

    print(f"Timestep {t} processed")

fig, ax = plt.subplots(1, 1, figsize = (4, 3))

ls = []
ln, = ax.plot(times[1:], R_s[1:], 'g*', markerfacecolor = "None", ms = 8, label = 'R')
ls.append(ln)

# ax2 = ax.twinx()
# ln, = ax2.plot(times[1:], radiis[1:, 0], 'rx', markerfacecolor = "None", ms = 8, label = 'a')
# ls.append(ln)
# ln, = ax2.plot(times[1:], radiis[1:, 1], 'bo', markerfacecolor = "None", ms = 8, label = 'b')
# ls.append(ln)
# ln, = ax2.plot(times[1:], radiis[1:, 2], 'ks', markerfacecolor = "None", ms = 8, label = 'c')
# ls.append(ln)


ax.set_xlabel("Timesteps")
ax.set_ylabel(r"$R_{mass}$")
ax.set_title(f"t = {t}")
# ax2.set_ylabel(r"$R_{PCA}$")

ax.legend(handles = ls)
ax.set_xscale("log")

R_ref = radiis[-1].copy()

In [ ]:
cutoff = 0
R_ref = np.array([18.17301002, 18.17301002, 18.17301002])

paths = sorted(glob.glob(f"{savedir}/{output_file}*.h5"))
slc = slice(60, len(paths))
paths = paths[slc]

times = np.zeros(len(paths))
radiis = np.zeros((len(paths), 3))

for i, path in enumerate(paths):
    t = int(path.split("_")[-1].split(".")[0])    
    times[i] = t

    profile = extract_data(path, boxDim, begin = 1, end = 2, nVars = 38)
    field = profile[0].copy()
    
    field = np.where(field > cutoff, 0, field)
    cm = center_of_mass(field)
    I = inertia_tensor(cm,field)
    m = droplet_mass(field)
    radiis[i] = radii_pca(eigenvalues(I), m) - R_ref
    
    print(f"Timestep {t} processed")

fig, ax = plt.subplots(1, 1, figsize = (4, 3))

marker_cols = ['rx', 'bo', 'ks']
labels = [r"$\delta_{x}$", r"$\delta_{y}$", r"$\delta_{z}$"]
for i in range(3):
    ax.plot(times, radiis[:, i], marker_cols[i], label = labels[i], markerfacecolor = 'none')

ax.set_xlabel("Timesteps")
ax.set_ylabel(r"$R_{PCA} - R_{ref}$")
ax.legend(ncol = 3)

sigma_fluct = droplet_fluctuations(radiis, temp = kbt) # y20 y22
print(r"$\sigma_{20}$="+f"{sigma_fluct[0]:.4f}" + r" $\sigma_{22}$="+f"{sigma_fluct[1]:.4f}" + r" $\sigma_{YL}$=0.017")
print(np.mean(sigma_fluct))

# Valid variable range

From the covariance matrix, certain limits on parameter values are set based upon ensuring that the diagonal remains positive. This limit is set by the term. $5 - c_s^2(k)$ in Xi[5, 5] which corresponds to a maximum allowable $c_s^2(k) = 0.\bar{5}$. The parameter that controls $c_s^2$ is $T$ as $c_s^2 = T$. $T_c$ or the critical temperature where demixing begins is defined as $T_c = \lambda/2$. In the expression for calculating $c_s^2(k) = c_s^2 + \kappa \rho_0 k^2$, $\kappa$ also controls the value of $c_s^2(k)$. Therefore this phase diagram will be defined using $\lambda$ and $\kappa$. Tested ranges will be $0.1 \leq \lambda \leq 1.1$ and $0.01 \leq \kappa \leq 0.05$

#### Helper functions

In [30]:
@njit
# Newton-Raphson method function
def newton_c_ver(initial_guess, chi = 1.1, T = 0.5, tolerance = 1e-4, max_iterations = 1000):
    x = initial_guess  # value of phi
    rho = 1.0

    for i in range(max_iterations):
        fx = calculate_df_dphi(rho, x, chi, T)
        dfx = calculate_dmup_dphi(rho, x, chi, T)

        # Prevent division by zero
        if dfx == 0.0:
            # print("Divide by 0 encountered in Newton Raphson")
            # sys.exit(-1)
            return None

        x_next = x - fx / dfx

        # Check if the difference between successive iterations is within tolerance
        if abs(x_next - x) < tolerance:
            return x_next

        x = x_next

    # print("Maximum iteration reached without convergence in Newton Raphson")
    return None

@njit()
def lattice_fourier_laplacian(kx, ky, kz):
    expr1 = np.cos(kx) + np.cos(ky) + np.cos(kz)
    expr2 = np.cos(kx)*np.cos(ky) + np.cos(ky)*np.cos(kz) + np.cos(kx)*np.cos(kz)
    out = 2/9*expr1 + 2/9*expr2 - 4/3
    return -out/(1/3)

@njit()
def calculate_dmup_drho(rho, phi, chi, T):
    out = -T*phi/(rho**2 - phi**2) + chi/2*phi/(rho**2)
    return out

@njit()
def calculate_dmup_dphi(rho, phi, chi, T):
    out = T*rho/(rho**2 - phi**2) - chi/(2*rho)
    return out

@njit()
def sound_speed_square(T):
    out = T
    return out

@njit()
def calculate_df_dphi(rho, phi, chi, T):
    out = -chi/2.*(phi/rho) + T/2.*np.log((1. + phi/rho)/(1. - phi/rho))
    return out

@njit()
def cholesky_decomp(arr_in, n, bstart):
    A = arr_in.copy()
    # sum = 0
    for i in range(bstart, n):
        for j in range(bstart, i + 1):
            sum = A[i*n + j]
            for k in range(j - 1, bstart - 1, -1):
                sum -= A[i*n+k]*A[j*n+k]
            if i == j:
                if sum >= 0:
                    A[i*n+j] = np.sqrt(sum)
                else:
                    A[i*n+j] = 0
                    raise ValueError(f"Row {i} in matrix not spd!")
            else:
                if A[j*n+j] > 0:
                    A[i*n+j] = sum/A[j*n+j]
                else:
                    raise ValueError("Matrix diagonal is 0")

    for i in range(0, n):
        for j in range(i + 1, n):
            A[i*n+j] = 0

    return A

@njit()
def covariance_matrix(rho0, phi0, k2, chi = 1.1, T = 0.5, kappa = 0.01, temperature = 1e-7, tau_r = 1, tau_p = 1, Gamma = 1):
    ndof = 38
    Q = ndof//2
    kT = temperature
    
    cs2 = sound_speed_square(T) + kappa*k2*rho0
    mu_rho = calculate_dmup_drho(rho0, phi0, chi, T)
    mu_phi = calculate_dmup_dphi(rho0, phi0, chi, T) + k2*kappa
    p_phi = k2*kappa*phi0

    lambdaLB_r = -1. / tau_r
    lambdaLB_p = -1. / tau_p

    lambda_r = -lambdaLB_r * (2 + lambdaLB_r) / 2
    lambda_p = -lambdaLB_p * (2 + lambdaLB_p) / 2
    lambda_rp = -lambdaLB_r * (2 + lambdaLB_p) / 2
    lambda_pr = -lambdaLB_p * (2 + lambdaLB_r) / 2

    Xi = np.zeros((ndof * ndof,), dtype=float)

    Xi[5 * ndof + 5] = 2. * Gamma * kT / rho0 * lambda_p
    Xi[6 * ndof + 6] = 2. * Gamma * kT / rho0 * lambda_p
    Xi[7 * ndof + 7] = 2. * Gamma * kT / rho0 * lambda_p
    Xi[8 * ndof + 8] = 2. * kT * rho0 * (5 - 9 * cs2) * lambda_r
    Xi[9 * ndof + 9] = 8. * kT * rho0 * lambda_r
    Xi[10 * ndof + 10] = (8.0 / 3.0) * kT * rho0 * lambda_r
    Xi[11 * ndof + 11] = (2.0 / 3.0) * kT * rho0 * lambda_r
    Xi[12 * ndof + 12] = (2.0 / 3.0) * kT * rho0 * lambda_r
    Xi[13 * ndof + 13] = (2.0 / 3.0) * kT * rho0 * lambda_r
    Xi[14 * ndof + 14] = 4. * kT * rho0 * lambda_r
    Xi[15 * ndof + 15] = 4. * kT * rho0 * lambda_r
    Xi[16 * ndof + 16] = 4. * kT * rho0 * lambda_r
    Xi[17 * ndof + 17] = (4.0 / 3.0) * kT * rho0 * lambda_r
    Xi[18 * ndof + 18] = (4.0 / 3.0) * kT * rho0 * lambda_r
    Xi[(Q + 0) * ndof + (Q + 0)] = (4.0 / 3.0) * kT * rho0 * lambda_r
    Xi[(Q + 1) * ndof + (Q + 1)] = 18. * kT * rho0 * (1 - cs2) * lambda_r
    Xi[(Q + 2) * ndof + (Q + 2)] = 8. * kT * rho0 * lambda_r
    Xi[(Q + 3) * ndof + (Q + 3)] = (8.0 / 3.0) * kT * rho0 * lambda_r
    Xi[(Q + 4) * ndof + (Q + 4)] = 2. * Gamma * kT / rho0 * (-9 * Gamma * mu_phi + 5) * lambda_p
    Xi[(Q + 5) * ndof + (Q + 5)] = 8. * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 6) * ndof + (Q + 6)] = (8.0 / 3.0) * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 7) * ndof + (Q + 7)] = (2.0 / 3.0) * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 8) * ndof + (Q + 8)] = (2.0 / 3.0) * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 9) * ndof + (Q + 9)] = (2.0 / 3.0) * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 10) * ndof + (Q + 10)] = 4. * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 11) * ndof + (Q + 11)] = 4. * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 12) * ndof + (Q + 12)] = 4. * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 13) * ndof + (Q + 13)] = (4.0 / 3.0) * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 14) * ndof + (Q + 14)] = (4.0 / 3.0) * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 15) * ndof + (Q + 15)] = (4.0 / 3.0) * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 16) * ndof + (Q + 16)] = 18. * Gamma * kT / rho0 * (-Gamma * mu_phi + 1) * lambda_p
    Xi[(Q + 17) * ndof + (Q + 17)] = 8. * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 18) * ndof + (Q + 18)] = (8.0 / 3.0) * Gamma * kT / rho0 * lambda_p

    Xi[8 * ndof + (Q + 1)] = 6. * kT * rho0 * (3 * cs2 - 1) * lambda_r
    Xi[(Q + 1) * ndof + 8] = 6. * kT * rho0 * (3 * cs2 - 1) * lambda_r

    Xi[(Q + 4) * ndof + (Q + 16)] = 6. * Gamma * kT / rho0 * (3 * Gamma * mu_phi - 1) * lambda_p
    Xi[(Q + 16) * ndof + (Q + 4)] = 6. * Gamma * kT / rho0 * (3 * Gamma * mu_phi - 1) * lambda_p

    Xi[8 * ndof + (Q + 4)] = -3. * kT * (Gamma * mu_phi * (rho0 ** 2) * (3 * cs2 - 1) * mu_rho)
    Xi[(Q + 4) * ndof + 8] = -3. * kT * (Gamma * mu_phi * (rho0 ** 2) * (3 * cs2 - 1) * mu_rho * lambda_pr + cs2 * (3 * Gamma * mu_phi - 1) * p_phi * lambda_rp) / (cs2 * mu_phi * rho0)
    Xi[(8) * ndof + (Q + 16)] = 3. * kT * (Gamma * mu_phi * (rho0 ** 2) * (3 * cs2 - 1) * mu_rho * lambda_pr + cs2 * (3 * Gamma * mu_phi - 1) * p_phi * lambda_rp) / (cs2 * mu_phi * rho0)
    Xi[(Q + 16) * ndof + 8] = 3. * kT * (Gamma * mu_phi * (rho0 ** 2) * (3 * cs2 - 1) * mu_rho * lambda_pr + cs2 * (3 * Gamma * mu_phi - 1) * p_phi * lambda_rp) / (cs2 * mu_phi * rho0)

    Xi[(Q + 1) * ndof + (Q + 4)] = 3. * kT * (Gamma * mu_phi * (rho0 ** 2) * (3 * cs2 - 1) * mu_rho * lambda_pr + cs2 * (3 * Gamma * mu_phi - 1) * p_phi * lambda_rp) / (cs2 * mu_phi * rho0)
    Xi[(Q + 4) * ndof + (Q + 1)] = 3. * kT * (Gamma * mu_phi * (rho0 ** 2) * (3 * cs2 - 1) * mu_rho * lambda_pr + cs2 * (3 * Gamma * mu_phi - 1) * p_phi * lambda_rp) / (cs2 * mu_phi * rho0)
    Xi[(Q + 1) * ndof + (Q + 16)] = -3. * kT * (Gamma * mu_phi * (rho0 ** 2) * (3 * cs2 - 1) * mu_rho * lambda_pr + cs2 * (3 * Gamma * mu_phi - 1) * p_phi * lambda_rp) / (cs2 * mu_phi * rho0)
    Xi[(Q + 16) * ndof + (Q + 1)] = -3. * kT * (Gamma * mu_phi * (rho0 ** 2) * (3 * cs2 - 1) * mu_rho * lambda_pr + cs2 * (3 * Gamma * mu_phi - 1) * p_phi * lambda_rp) / (cs2 * mu_phi * rho0)

    Xi[(0) * ndof + (Q + 4)] = -3. * Gamma * kT * rho0 * mu_rho / cs2 * lambda_pr
    Xi[(Q + 4) * ndof + (0)] = -3. * Gamma * kT * rho0 * mu_rho / cs2 * lambda_pr
    Xi[(0) * ndof + (Q + 16)] = 3. * Gamma * kT * rho0 * mu_rho / cs2 * lambda_pr
    Xi[(Q + 16) * ndof + (0)] = 3. * Gamma * kT * rho0 * mu_rho / cs2 * lambda_pr

    Xi[(1) * ndof + (Q + 1)] = 3. * kT * p_phi / (mu_phi * rho0) * lambda_rp
    Xi[(Q + 1) * ndof + (1)] = 3. * kT * p_phi / (mu_phi * rho0) * lambda_rp
    Xi[(1) * ndof + (8)] = -3. * kT * p_phi / (mu_phi * rho0) * lambda_rp
    Xi[(8) * ndof + (1)] = -3. * kT * p_phi / (mu_phi * rho0) * lambda_rp
    Xi[(2) * ndof + (5)] = -phi0 * kT * lambda_pr
    Xi[(3) * ndof + (6)] = -phi0 * kT * lambda_pr
    Xi[(4) * ndof + (7)] = -phi0 * kT * lambda_pr
    Xi[(5) * ndof + (2)] = -phi0 * kT * lambda_pr
    Xi[(6) * ndof + (3)] = -phi0 * kT * lambda_pr
    Xi[(7) * ndof + (4)] = -phi0 * kT * lambda_pr

    return Xi

In [164]:
@njit
def output_spd_homogenous(k2, CHI, KAPPA, T_IN):
    SPD = np.zeros_like((CHI))

    for x in range(SPD.shape[0]):
        for y in range(SPD.shape[1]):
            for z in range(SPD.shape[2]):
                test_spd = True
                for kval in np.nditer(k2.T):
                    try:
                        Xi = covariance_matrix(1, 0, kval, chi = CHI[x,y,z], T = T_IN[x,y,z], kappa = KAPPA[x, y, z])
                        cholesky_decomp(Xi, 38, 5)
                    except:
                        test_spd = False
                        break 
                SPD[x, y, z] = test_spd
                
    return SPD

@njit
def pre_process_inhomogenous_spatially_independent(chi, T, kappa):
    # guess = np.sqrt(3*(chi/2 - T)/T)*1/(np.sqrt(2))
    phi0_guess = np.arange(0.4, 0.8, 0.1)
    for guess in phi0_guess:
        phi0 = newton_c_ver(guess, chi = chi, T = T, tolerance = 1e-4, max_iterations = 1000)
        if phi0 is not None:
            break
    print(phi0)
    # print("phi0:",phi0, " T:",T, " chi:",chi, " kappa:",kappa)

    xi = np.sqrt((0.142*kappa)/((chi/2 - T)-0.31*T*(phi0**2)))
    
    fit_func = lambda x, b:phi0*np.tanh(0.76*(x - b)/xi)
    xraw = np.linspace(-6, 6, 13)
    phi_s = fit_func(xraw, -0.5)
    return phi_s

@njit
def output_spd_inhomogenous_spatially_independent(CHI, T_IN, KAPPA):
    SPD = np.zeros_like((CHI))
    test_spd = True

    for x in range(SPD.shape[0]):
        for y in range(SPD.shape[1]):
            for z in range(SPD.shape[2]):
                chi = CHI[x,y,z]
                T = T_IN[x,y,z]
                kappa = KAPPA[x,y,z]
                phi_s = pre_process_inhomogenous_spatially_independent(chi, T, kappa)
                for phi in np.nditer(phi_s):
                    try:
                        Xi = covariance_matrix(1, phi, 0, chi = chi, T = T, kappa = kappa, tau_r = 0.7886751345948129)
                        cholesky_decomp(Xi, 38, 5)
                    except:
                        test_spd = False
            
                SPD[x,y,z] = test_spd

    return SPD

In [ ]:
chi = 0.6
T = 0.3*chi
kappa = 0.0144

phi_s = pre_process_inhomogenous_spatially_independent(chi, T, kappa)
test_spd = True
for phi in np.nditer(phi_s):
    try:
        Xi = covariance_matrix(1, phi, 0, chi = chi, T = T, kappa = kappa, tau_r = 0.7886751345948129)
        cholesky_decomp(Xi, 38, 5)
    except:
        test_spd = False

test_spd

#### Plotting

In [140]:
def make_plot_homogenous(points = 10, L = 16):
    chimin = 0.2
    chimax = 1.2

    kappamin = 0.0
    kappamax = 0.05

    propmin = 0.5
    propmax = 0.8

    chi_s = np.linspace(chimin, chimax, points)
    kappa_s = np.linspace(kappamin, kappamax, points)
    prop_s = np.linspace(propmin, propmax, points)
    # prop_s[points//2] = 0.5

    CHI, KAPPA, PROP = np.meshgrid(*[chi_s, kappa_s, prop_s])
    T_IN = CHI*PROP

    freqs = fft.fftshift(fft.fftfreq(L))
    kx, ky, kz = np.meshgrid(*tuple([2*np.pi*freqs for L in [L, L, L]]), indexing='ij')
    k2 = lattice_fourier_laplacian(kx, ky, kz)
    
    SPD = output_spd_homogenous(k2, CHI, KAPPA, T_IN)
    np.savez("spd_covariance_homogenous_matrix.npz", chi = CHI, kappa = KAPPA, T = T_IN, SPD = SPD)

    return 1

In [ ]:
import os

if not os.path.exists("spd_covariance_homogenous_matrix.npz"):
    make_plot_homogenous(points = 50)

# make_plot(points = 10)
# make_plot_homogenous(points = 10)

FILE_IN = np.load("spd_covariance_homogenous_matrix.npz")
CHI = FILE_IN['chi']
KAPPA = FILE_IN['kappa']
T_IN = FILE_IN['T']
SPD = FILE_IN['SPD']
PROPS = T_IN/CHI

fig = plt.figure(figsize = (6, 6)) 
ax = fig.add_subplot(projection='3d')
ax.set_proj_type('ortho')
colors = np.empty(SPD.shape, dtype=object)
colors[SPD == 0] = "w"
colors[SPD == 1] = "lime"

out = ax.voxels(CHI, T_IN, KAPPA, SPD[:-1, :-1, :-1], facecolors=colors[:-1, :-1, :-1], edgecolor='k')
ax.set_xlabel(r"$\lambda$")
ax.set_ylabel(r"$T_{in}$")
ax.set_zlabel(r"$\kappa$")
fig.tight_layout()

# ax.view_init(30, 60, 0) # ax.view_init(elev, azim, roll)
ax.view_init(30, 60, 0) # ax.view_init(elev, azim, roll)

ax.set_title("Homogenous system")
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (4, 4))

slc_plot = np.s_[10, :, :]
im = ax.contourf(CHI[slc_plot], PROPS[slc_plot], SPD[slc_plot], levels = 1, colors = ['tab:red', 'tab:green'])
ax.set_xlabel(r"$\chi$")
ax.set_ylabel(r"$T/\chi$")
curr_prop = np.unique(KAPPA[slc_plot])[0]
ax.set_title(f"$\kappa = {curr_prop:.3f}$")

fig.tight_layout()
fig.savefig(f"figures/valid_homogenous_parameters.png", dpi = 300)

In [156]:
def make_plot_inhomogenous(points = 10, L = 16):
    chimin = 0.4
    chimax = 0.8

    kappamin = 0.01
    kappamax = 0.05

    propmin = 0.35
    propmax = 0.49

    chi_s = np.linspace(chimin, chimax, points)
    kappa_s = np.linspace(kappamin, kappamax, points)
    prop_s = np.linspace(propmin, propmax, points)

    CHI, KAPPA, PROP = np.meshgrid(*[chi_s, kappa_s, prop_s])
    T_IN = CHI*PROP

    SPD = output_spd_inhomogenous_spatially_independent(CHI, T_IN, KAPPA)
    np.savez("spd_covariance_inhomogenous_matrix.npz", chi = CHI, kappa = KAPPA, T = T_IN, SPD = SPD)

    return 1

In [ ]:
import os

# if not os.path.exists("spd_covariance_inhomogenous_matrix.npz"):
    # make_plot_inhomogenous(points = 10)

make_plot_inhomogenous(points = 10)
FILE_IN = np.load("spd_covariance_inhomogenous_matrix.npz")
CHI = FILE_IN['chi']
KAPPA = FILE_IN['kappa']
T_IN = FILE_IN['T']
SPD = FILE_IN['SPD']
PROPS = T_IN/CHI

fig = plt.figure(figsize = (6, 6)) 
ax = fig.add_subplot(projection='3d')
ax.set_proj_type('ortho')
colors = np.empty(SPD.shape, dtype=object)
colors[SPD == 0] = "w"
colors[SPD == 1] = "lime"

out = ax.voxels(CHI, T_IN, KAPPA, SPD[:-1, :-1, :-1], facecolors=colors[:-1, :-1, :-1], edgecolor='k')
ax.set_xlabel(r"$\lambda$")
ax.set_ylabel(r"$T_{in}$")
ax.set_zlabel(r"$\kappa$")

# ax.view_init(30, 60, 0) # ax.view_init(elev, azim, roll)
ax.view_init(30, 60, 0)

ax.set_title("Inhomogenous system")
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (3, 3))

slc_plot = np.s_[1, :, :]
im = ax.contourf(CHI[slc_plot], PROPS[slc_plot], SPD[slc_plot], levels = 1, colors = ['tab:red', 'tab:green'])
ax.set_xlabel(r"$\lambda$")
ax.set_ylabel(r"$T/\lambda$")
curr_prop = np.unique(KAPPA[slc_plot])[0]
ax.set_title(f"$\kappa = {curr_prop:.4f}$")

fig.tight_layout()

In [ ]:
SPD[0, :, :]